In [ ]:
import os
print(os.getcwd())

d:\공모전\25-1 비어플\빈집\전처리_통합


데이터는 현재 작업폴더속의 원본데이터, 결과데이터 하위폴더에 각각 보관돼있고 저장된다.

# 데이터 전처리 총집합

1. 빈집정보알림e 빈집데이터 추출
2. 전국 편의점 정보
3. 전국 시장 정보
4. 전국 영화관 정보
5. 전국 병원 정보
6. 전국 학교 정보
7. 전국 공장 정보
8. 전국 학원 정보
9. 전국 경찰서 정보
10. 전국 공원 정보
11. 전국 버스정류장 정보
12. 전국 경로당&마을회관 정보
13. 전국 지하철역 정보
14. 전국 기차역 정보
15. 전국 고속버스터미널 정보
16. 전국 은행 정보
17. 전국 농기계대여소 정보
18. 전국 농산물 직거래 판매장 정보
19. 전국 차량정비소 정보
20. 전국 읍,면,동사무소 정보
21. 전국 요양원 정보
22. 전국 농업기술센터, 행정복지센터 정보
23. 전국 우체국 정보  
---
24. 전국 **공시지가, 실거래가** 정보
25. 전국 농지매물 정보
26. 위도경도 거리 계산, 반경 시설 개수 계산

# 패키지 불러오기

In [3]:
import pandas as pd
import numpy as np
import math

import time
#크롤링용
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
import requests

#네이버API
from concurrent.futures import ThreadPoolExecutor

# 사용자 지정 함수

## 1) 주소 분할 - 사분할

주로 지번주소일 경우

In [4]:
# 네개로 분할하는 함수
def split_full_address(addr):
    if isinstance(addr, str):  # 문자열일 때만 처리
        parts = addr.strip().split()
        if len(parts) >= 4:
            return pd.Series([parts[0], parts[1], parts[2], " ".join(parts[3:])])
        elif len(parts) == 3:
            return pd.Series([parts[0], parts[1], parts[2], pd.NA]) #읍면동까지
        elif len(parts) == 2:
            return pd.Series([parts[0], parts[1], pd.NA, pd.NA]) #시군구까지
        else:
            return pd.Series([pd.NA, pd.NA, pd.NA, addr]) #나머지주소만 (기타)
    else:
        return pd.Series([pd.NA, pd.NA, pd.NA, pd.NA]) #주소가 없을 경우

## 2) 주소 분할 - 삼분할

주로 도로명 주소일 경우

In [5]:
# 세개로 분할하는 함수
def split_full_address_3(addr):
    if isinstance(addr, str):  # 문자열일 때만 처리
        parts = addr.strip().split()
        if len(parts) >= 3:
            return pd.Series([parts[0], parts[1], " ".join(parts[2:])])
        elif len(parts) == 2:
            return pd.Series([parts[0], parts[1], pd.NA]) #시군구까지
        else:
            return pd.Series([pd.NA, pd.NA, addr]) #나머지주소만 (기타)
    else:
        return pd.Series([pd.NA, pd.NA, pd.NA]) #주소가 없을 경우

## 3) 네이버 지도 API 좌표

API키가 입력되있는 상태로, 금액이 청구될 수 있으니 주의해주세요

In [6]:
# NAVER API 키
API_KEY_ID = 'xatfhyujvl'        # API Key ID
API_KEY = '1lZJ61ePs88EWUaFqFH48xr7VQBvrJU8Il13GjrI'  # API Key

# 주소를 받아 위도, 경도 정보를 반환하는 함수
def get_coordinates_from_address(address):
    base_url = "https://maps.apigw.ntruss.com/map-geocode/v2/geocode"
    params = {'query': address}
    headers = {
        'X-NCP-APIGW-API-KEY-ID': API_KEY_ID,
        'X-NCP-APIGW-API-KEY': API_KEY,
        'Accept': 'application/json'
    }
    
    try:
        response = requests.get(base_url, headers=headers, params=params)
        if response.status_code == 200:
            data = response.json()
            if data.get('addresses'):
                addr_info = data['addresses'][0]
                return {'Latitude': addr_info.get('y', ''), 'Longitude': addr_info.get('x', '')}
    except Exception as e:
        print(f"Error for address {address}: {e}")
    
    # 실패 시 결측치 값 반환
    return {'Latitude': pd.NA, 'Longitude': pd.NA}

아래의 코드를 API를 사용할때마다 복사, 수정후 사용

In [ ]:
# #주소 칼럼 입력
# addresses = data['주소'].tolist()

# # ThreadPoolExecutor를 이용해 동시 실행
# ##하나씩 실행하면 느리다
# with ThreadPoolExecutor(max_workers=100) as executor: #한번에 100개
#     results = list(executor.map(get_coordinates_from_address, addresses))

# # 결과를 DataFrame으로 변환 후 기존 DataFrame과 결합
# df_coordinates = pd.DataFrame(results)
# df_final = pd.concat([data.reset_index(drop=True), df_coordinates], axis=1)
# print(df_final)

# 1. 빈집정보알림e 빈집데이터 추출

## 현재 이 사이트가 개편이 되어서 데이터를 더이상 추출할수가 없다.

http://binzibe.kr/binzib/binzibplatform/statisticInfo/binzibPublicInfoStatus.do  
위의 사이트에서 빈집의 주소 목록을 동적 웹 크롤링을 통해 추출한다.  
자세한 번지수는 기록되어있지 않으므로, 행정동의 최소 단위인 읍면동, 또는 리까지 있음에 유의하자.

## 크롤링 하기

1. http://binzibe.kr/binzib/binzibplatform/statisticInfo/binzibPublicInfoStatus.do
2. 동적웹크롤링으로 데이터 추출하기, 추출한 데이터는 데이터프레임으로 저장
3. #content > section > div > div.left > div > div.contBtm > ul.contResult > li:nth-child(1) > ul > li:nth-child(1) > span의 정보를 텍스트로 불러오고 "소재지" 열에 저장
4. #content > section > div > div.left > div > div.contBtm > ul.contResult > li:nth-child(1) > ul > li:nth-child(2) > span의 정보를 텍스트로 불러오고 "주택유형"열에 저장
5. #content > section > div > div.left > div > div.contBtm > ul.contResult > li:nth-child(1) > ul > li:nth-child(3) > spand의 정보를 텍스트로 불러오고 "등급"열에 저장
6. #content > section > div > div.left > div > div.contBtm > ul.contResult > li:nth-child(1) > ul > li:nth-child(4) > span의 정보를 텍스트로 불러오고 "용도지역"열에 저장
7. #content > section > div > div.left > div > div.contBtm > ul.contResult > li:nth-child(20) > ul > li:nth-child(1) > span 이것처럼 한 페이지에 대략 20개의 정보가 들어있다. 이 20개의 정보를 동일한 과정을 통해 추출한다. 하지만 한 페이지에 정보가 20개보다 많을수도 적을수도 있다. 이런 예외사항이 있는 경우를 해결.
8. 다 불러왔으면 다음 페이지로 넘어간다. #pagination-container > div > div > ul > li.paginationjs-next.J-paginationjs-next > a 이 버튼을 누르면 다음페이지로 넘어간다. 그리고 위의 작업을 반복한다. 그 페이지에서 작업이 다 끝났으면 다음 페이지로 넘어가기를 반복한다.
9. 도중에 에러가 발생하여 코드가 작동이 멈출 수 있다. 따라서 10페이지 단위로 작업을 실행한다. 처음 10페이지까지 작업을 마쳤으면 이것을 csv파일로 저장, 이름은 temp_data_10.csv로 한다. 이후 11~20페이지까지 진행한 후 이 데이터를 temp_data_20.csv로 한다. 이 과정을 6434페이지까지 반복한다.(이 페이지가 마지막 페이지이다.) 코드가 다 실행이 된 후에 마지막으로 csv파일을 만드는 것이 아니라 코드가 실행되는 중간에 파일을 만들 수 있도록 한다. 이렇게 해야 에러가 나더라도 지금까지 만든 파일을 보고 다시 에러가 일어났던 페이지부터 작업을 반복할 수 있다.
10. 따라서 내가 직접 시작할 페이지를 입력하는 기능도 넣는다. 하지만 내가 원하는 페이지를 한번에 이동할 수 없고 8번에 언급한 다음페이지로 넘어가는 버튼을 하나하나 누를 수 밖에 없다. 15페이지로 갈거면 현제 1페이지에서 버튼을 14번 눌러여 한다. 이것도 반영하여 코드를 작성한다.
11. WebDriverWait함수를 사용하여 페이지가 빨리 로딩이 되지 않는 경우에 대처할 수 있게 한다.

### 작업에 320분이 걸리므로 함부로 실행하지 말것

In [ ]:
# # 시작 페이지 번호와 마지막 페이지 번호
# start_page = int(input("시작 페이지 번호: ")) #1페이지지
# max_page = int(input("마지막 페이지 번호: ")) #3월 기준 6535페이지까지 존재

# # Chrome WebDriver
# driver = webdriver.Chrome()

# # URL 접속
# url = "http://binzibe.kr/binzib/binzibplatform/statisticInfo/binzibPublicInfoStatus.do"
# driver.get(url)

# # 콘텐츠 및 레코드 요소
# content_locator = (By.CSS_SELECTOR, "#content > section > div > div.left > div > div.contBtm > ul.contResult")
# record_locator = "#content > section > div > div.left > div > div.contBtm > ul.contResult > li"

# # 초기 콘텐츠 로딩 대기
# WebDriverWait(driver, 20).until(EC.presence_of_element_located(content_locator))
# WebDriverWait(driver, 20).until(lambda d: len(d.find_elements(By.CSS_SELECTOR, record_locator)) > 0)

# # 시작 페이지까지 이동 (현재 페이지 번호는 버튼을 클릭한 횟수로 기록)
# current_page = 1
# while current_page < start_page:
#     next_buttons = driver.find_elements(By.CSS_SELECTOR,
#         "#pagination-container > div > div > ul > li.paginationjs-next.J-paginationjs-next > a")
#     if next_buttons:
#         next_btn = next_buttons[0]
#         next_btn.click()
#         time.sleep(0.7)  # 0.7초 대기 후 페이지 정보 추출
#         current_page += 1
#     else:
#         print("오류 발생")
#         driver.quit()
#         exit()

# data_list = []

# while current_page <= max_page:
#     print(f"\n현재 페이지: {current_page}") #잘 진행되고있는지 확인용
#     try:
#         # 페이지 내 모든 요소들이 로드될 때까지 대기
#         WebDriverWait(driver, 20).until(EC.presence_of_element_located(content_locator))
#         WebDriverWait(driver, 20).until(lambda d: len(d.find_elements(By.CSS_SELECTOR, record_locator)) > 0)
#     except Exception as e:
#         print(f"페이지 {current_page} 로딩 중 에러 발생: {e}")

#     record_elements = driver.find_elements(By.CSS_SELECTOR, record_locator)
    
#     for record in record_elements:
#         try:
#             inner_ul = record.find_element(By.TAG_NAME, "ul")
#             location = inner_ul.find_element(By.CSS_SELECTOR, "li:nth-child(1) > span").text.strip() #주소
#             house_type = inner_ul.find_element(By.CSS_SELECTOR, "li:nth-child(2) > span").text.strip() #주택유형
#             grade = inner_ul.find_element(By.CSS_SELECTOR, "li:nth-child(3) > span").text.strip() #주택 등급
#             usage_area = inner_ul.find_element(By.CSS_SELECTOR, "li:nth-child(4) > span").text.strip() #용도지역
#         except Exception as inner_e:
#             location, house_type, grade, usage_area = None, None, None, None #에러가 발생했거나 내용이 없는 경우
        
#         data_list.append({
#             "소재지": location,
#             "주택유형": house_type,
#             "등급": grade,
#             "용도지역": usage_area
#         })
    
#     # 10페이지 단위로 CSV 파일 저장
#     # 중간에 오류가 나더라도 이어서 진행할 수 있도록 (파일이 총 페이지수 / 10 개가 생성되므로 주의하자)
#     if current_page % 10 == 0:
#         df = pd.DataFrame(data_list)
#         filename = f"temp_data_{current_page}.csv"
#         df.to_csv(filename, index=False, encoding='utf-8-sig')
    
#     # 다음 페이지 버튼을 클릭하고 1.5초 후 다음 페이지 정보 추출 (사이트 로딩이 되기를 기다힘)
#     next_buttons = driver.find_elements(By.CSS_SELECTOR,
#         "#pagination-container > div > div > ul > li.paginationjs-next.J-paginationjs-next > a")
#     if next_buttons:
#         next_btn = next_buttons[0]
#         next_btn.click()
#         time.sleep(1.5)  # 다음 페이지 로딩을 위해 1.5초 대기
#         current_page += 1
#     else:
#         print(f"마지막 {current_page}페이지 이후 내용이 없어 종료")
#         break

# driver.quit()

최종적으로 653개의 csv파일, 총 3.7기가의 csv파일 생성  
오류없이 잘 실행되었다면 temp_data_6530.csv에 모든 데이터가 들어있을 것

## 전처리를 위해 최종 데이터 불러오기

### 사이트가 개편되기전 추출한 전체 데이터이다.

In [ ]:
data=pd.read_csv("원본데이터/temp_data_6530.csv", encoding='utf-8-sig')
data.head()

### 소재지 주소 전처리

In [ ]:
# 주소 분리 함수
def parse_address(address):
    parts = address.split()
    if len(parts) == 4: #4개로 분리된다면
        시도, 시군구, 읍면동, 나머지 = parts
        리 = pd.NA
    elif len(parts) == 5:
        시도, 시군구, 읍면동, 리, 나머지 = parts #5개로 분리된다면
    else:
        # 주소가 없다면
        시도 = 시군구 = 읍면동 = 리 = 나머지 = pd.NA
    return pd.Series([시도, 시군구, 읍면동, 리, 나머지])
data[['시도', '시군구', '읍면동', '리', '나머지']] = data['소재지'].apply(parse_address)
data.head()

In [ ]:
#데이터 칼럼 순서 바꾸기
data=data[["시도","시군구","읍면동","리","주택유형","등급","용도지역"]]

In [ ]:
#컬럼 영문명으로 바꾸기
data.columns=["County","City","Town","Village","Housing_Type","Grade","Zoning"]

## 최종 데이터 저장

In [ ]:
data.isna().sum()

결측치도 Village(리)에만 존재하니 아무 문제 없다

In [ ]:
data.to_csv("결과데이터/빈집데이터_빈집정보알림e.csv",index=False,encoding='utf-8-sig')

## 추가 : 호남, 영남지방 데이터만 추출

In [6]:
data.County.unique()

array(['대구광역시', '인천광역시', '광주광역시', '부산광역시', '경상북도', '서울특별시', '대전광역시',
       '충청북도', '전북특별자치도', '전라남도', '충청남도', '경상남도', '경기도', '울산광역시',
       '제주특별자치도', '강원특별자치도', '세종특별자치시'], dtype=object)

대구광역시, 광주광역시, 부산광역시, 경상북도, 전북특별자치도, 전라남도, 경상남도, 울산광역시 지역만 추출하자

In [7]:
#지역 리스트
target_counties = ['대구광역시', '광주광역시', '부산광역시','경상북도', '전북특별자치도', '전라남도','경상남도', '울산광역시']
data_filtered = data[data['County'].isin(target_counties)].reset_index(drop=True)
data_filtered.head()

,County,City,Town,Village,Housing_Type,Grade,Zoning
0,대구광역시,달서구,감삼동,NaN,다가구주택,2등급(구),제2종일반주거지역
1,대구광역시,달서구,두류동,NaN,단독주택,1등급(구),제2종일반주거지역
2,대구광역시,달서구,성당동,NaN,단독주택,1등급(구),제2종일반주거지역
3,대구광역시,달서구,두류동,NaN,단독주택,2등급(구),제2종일반주거지역
4,대구광역시,달서구,월암동,NaN,단독주택,4등급(구),제3종일반주거지역


In [9]:
data_filtered.County.unique()

array(['대구광역시', '광주광역시', '부산광역시', '경상북도', '전북특별자치도', '전라남도', '경상남도',
       '울산광역시'], dtype=object)

잘 추출되었다.

## 네이버 지도 API

각 주소에 대응되는 좌표 데이터 결합

In [24]:
data_filtered["전체주소"]=data_filtered["County"]+" "+data_filtered["City"]+" "+data_filtered["Town"]+" "+data_filtered["Village"].fillna("")
data_filtered.head(1)

,County,City,Town,Village,Housing_Type,Grade,Zoning,전체주소
0,대구광역시,달서구,감삼동,NaN,다가구주택,2등급(구),제2종일반주거지역,대구광역시 달서구 감삼동


API용 "전체주소"컬럼 생성

In [26]:
# 기존 DataFrame의 '전체주소' 컬럼을 리스트로 변환
addresses = data_filtered['전체주소'].tolist()

# ThreadPoolExecutor를 이용해 동시 실행
with ThreadPoolExecutor(max_workers=100) as executor:
    results = list(executor.map(get_coordinates_from_address, addresses))

# 결과를 DataFrame으로 변환 후 기존 DataFrame과 결합
df_coordinates = pd.DataFrame(results)
df_final = pd.concat([data_filtered.reset_index(drop=True), df_coordinates], axis=1)
print(df_final)

        County City Town Village Housing_Type   Grade     Zoning  \
0        대구광역시  달서구  감삼동     NaN        다가구주택  2등급(구)  제2종일반주거지역   
1        대구광역시  달서구  두류동     NaN         단독주택  1등급(구)  제2종일반주거지역   
2        대구광역시  달서구  성당동     NaN         단독주택  1등급(구)  제2종일반주거지역   
3        대구광역시  달서구  두류동     NaN         단독주택  2등급(구)  제2종일반주거지역   
4        대구광역시  달서구  월암동     NaN         단독주택  4등급(구)  제3종일반주거지역   
...        ...  ...  ...     ...          ...     ...        ...   
84801  전북특별자치도  정읍시  입암면     연월리         단독주택    특정빈집          -   
84802  전북특별자치도  정읍시  입암면     연월리         단독주택    일반빈집          -   
84803  전북특별자치도  정읍시  입암면     연월리         단독주택    일반빈집          -   
84804  전북특별자치도  정읍시  입암면     연월리         단독주택    일반빈집          -   
84805  전북특별자치도  정읍시  입암면     연월리         단독주택    특정빈집          -   

                      전체주소   Latitude   Longitude  
0           대구광역시 달서구 감삼동   35.849186  128.540694  
1           대구광역시 달서구 두류동     35.8541    128.5617  
2           대구광역시 달서구 성당동  

In [34]:
df_final.isna().sum()

County              0
City                0
Town                0
Village         32250
Housing_Type        0
Grade               0
Zoning              0
전체주소                0
Latitude            2
Longitude           2
dtype: int64

두개의 좌표빼고 전부 들었다. 구글 지도를 통해 수작업으로 추가하자

In [33]:
df_final[["Latitude","Longitude"]]=df_final[["Latitude","Longitude"]].apply(pd.to_numeric, errors='coerce')

좌표를 숫자형으로 전환

In [44]:
df_final[df_final['Latitude'].isna() | df_final['Longitude'].isna()]

,County,City,Town,Village,Housing_Type,Grade,Zoning,전체주소,Latitude,Longitude
28528,대구광역시,달서구,상인동,NaN,단독주택,3등급(구),제2종일반주거지역,대구광역시 달서구 상인동,NaN,NaN
44280,경상북도,예천군,용문면,두천리,단독주택,2등급(신),계획관리지역,경상북도 예천군 용문면 두천리,NaN,NaN


In [46]:
df_final.loc[28528, ["Latitude", "Longitude"]]=[35.813698, 128.554675]
df_final.loc[44280, ["Latitude", "Longitude"]]=[36.701922, 128.348774]
df_final.loc[[28528, 44280]]

,County,City,Town,Village,Housing_Type,Grade,Zoning,전체주소,Latitude,Longitude
28528,대구광역시,달서구,상인동,NaN,단독주택,3등급(구),제2종일반주거지역,대구광역시 달서구 상인동,35.813698,128.554675
44280,경상북도,예천군,용문면,두천리,단독주택,2등급(신),계획관리지역,경상북도 예천군 용문면 두천리,36.701922,128.348774


## 추출된 데이터 저장

In [47]:
df_final.drop(columns=["전체주소"], inplace=True, axis=1)

In [ ]:
df_final.to_csv("결과데이터/빈집데이터_빈집정보알림e_영남호남지역.csv",index=False,encoding='utf-8-sig')

# 2. 전국 편의점 정보

http://www.safemap.go.kr/openApiService/data/getConvenienceStoreData.do  
공공데이터 포털에서 API를 이용해 불러온다  
인증키 : HNPHPFLS-HNPH-HNPH-HNPH-HNPHPFLS1K  

## 편의점 API 실행

In [ ]:
#설정값
service_key = "HNPHPFLS-HNPH-HNPH-HNPH-HNPHPFLS1K"  # 인증키
base_url = "http://www.safemap.go.kr/openApiService/data/getConvenienceStoreData.do"
fclty_cd = "509010"  # 편의점 필터 코드
num_of_rows = 100  # 한 페이지 최대 100개의 자료
data_type = "JSON"

In [ ]:
# 전체 페이지 개수 조회
def get_total_count():
    params = {
        "serviceKey": service_key,
        "numOfRows": 1,
        "pageNo": 1,
        "Fclty_Cd": fclty_cd,
        "dataType": data_type
    }
    response = requests.get(base_url, params=params)
    data = response.json()
    return int(data["response"]["body"]["totalCount"])

전체 페이지 개수를 모르기에 이를 추출하는 함수를 정의

In [ ]:
# 데이터 수집
failed_pages=[] #불러오기 실패한 데이터의 목록 리스트

def fetch_all_data():
    total_count = get_total_count()
    total_pages = math.ceil(total_count / num_of_rows)
    results = []

    for page in range(1, total_pages + 1):
        try:
            params = {
                "serviceKey": service_key,
                "numOfRows": num_of_rows,
                "pageNo": page,
                "Fclty_Cd": fclty_cd,
                "dataType": data_type
            }
            response = requests.get(base_url, params=params)
            data = response.json()

            result_code = data["response"]["header"]["resultCode"]
            if result_code != "00":
                raise Exception(data["response"]["header"].get("resultMsg", "Unknown error"))

            items = data["response"]["body"].get("items", [])
            for item in items:
                results.append({
                    "FCLTY_TY": item.get("FCLTY_TY"),
                    "FCLTY_NM": item.get("FCLTY_NM"),
                    "ADRES": item.get("ADRES")
                })

            print(f"✅ {page}/{total_pages} 페이지 완료")
        
        except Exception as e:
            print(f"{page}번째 페이지 실패: {e}")
            failed_pages.append(page)

    return pd.DataFrame(results)


In [15]:
# API 실행
df_main = fetch_all_data()

NameError: name 'fetch_all_data' is not defined

## 실패한 페이지 재탐색

In [ ]:
failed_pages

In [ ]:
params = {
    "serviceKey": service_key,
    "numOfRows": 100,
    "pageNo": 562, #페이지 번호를 바꿔가며 살펴보기
    "Fclty_Cd": fclty_cd,
    "dataType": "JSON"
}
response = requests.get(base_url, params=params)
print(response.json())

이 함수는 각 페이지별로 불러온 내용들을 표시한다.  
562페이지에는 데이터가 다 있고, 563페이지부터 데이터가 계속 없다.  
따라서 562페이지까지 데이터를 불러오면 된다

## 데이터 전처리 및 저장

### 주소의 분리 (도로명주소)

In [ ]:
def split_full_address_1(addr):
    if isinstance(addr, str):  # 문자열일 때만 처리
        parts = addr.strip().split()
        return pd.Series([parts[0], parts[1], parts[2], " ".join(parts[3:])])
    else:
        return pd.Series([pd.NA, pd.NA, pd.NA, pd.NA])  # 주소가 없을 경우
df_main[["시도", "시군구", "읍면동", "상세주소"]] = df_main["ADRES"].apply(split_full_address_1)

In [ ]:
df_main.isna().sum()

주소에 결측치는 없다

In [ ]:
df_main[["시도", "시군구", "읍면동", "상세주소"]] = df_main["ADRES"].apply(split_full_address)

### 데이터를 보기 쉽게

In [ ]:
df_main.rename(columns={"FCLTY_TY": "업종", "FCLTY_NM": "업소명"}, inplace=True)
df_main=df_main[["업종","업소명","시도","시군구","읍면동","상세주소"]]

## 네이버 지도 API

각 주소에 대응되는 위도, 경도 데이터를 결합하자

In [ ]:
df_main["전체주소"]=df_main["시도"]+" "+df_main["시군구"]+" "+df_main["읍면동"]+" "+df_main["상세주소"]
df_main.head(1)

API추출용 "전체주소"칼럼 생성

In [ ]:
# 기존 DataFrame의 '전체주소' 컬럼을 리스트로 변환
addresses = df_main['전체주소'].tolist()

# ThreadPoolExecutor를 이용해 동시 실행
with ThreadPoolExecutor(max_workers=100) as executor:
    results = list(executor.map(get_coordinates_from_address, addresses))

# 결과를 DataFrame으로 변환 후 기존 DataFrame과 결합
df_coordinates = pd.DataFrame(results)
df_final = pd.concat([df_main.reset_index(drop=True), df_coordinates], axis=1)
print(df_final)

In [ ]:
df_final.drop("전체주소", axis=1, inplace=True)
df_final.head()

## 컬럼 영문명으로 전환

In [ ]:
df_final.columns=["Business_Type","Conve_Name","County","City","Town","Address","Latitude","Longitude"]
df_final.dropna(subset=["County"],inplace=True)
df_final.head(2)

## 데이터 저장

In [ ]:
df_final.to_csv("결과데이터/편의점_목록_2024.csv", index=False, encoding="cp949")

# 3. 전국 시장 정보

In [ ]:
data=pd.read_excel("원본데이터/08_25_01_P_대규모점포.xlsx") #엑셀파일이다
data.head(1)

c:\ProgramData\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,번호,개방서비스명,개방서비스아이디,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태구분코드,영업상태명,상세영업상태코드,...,도로명전체주소,도로명우편번호,사업장명,최종수정시점,데이터갱신구분,데이터갱신일자,업태구분명,좌표정보X(EPSG5174),좌표정보Y(EPSG5174),점포구분명
0,1,대규모점포,08_25_01_P,3240000,2025324028907500004,2025-03-11,NaN,1,영업/정상,5,...,NaN,NaN,(주)이마트에브리데이 둔촌점,2025-03-11 09:32:28,I,2025-03-13 00:11:06,구분없음,212027.481159,446598.591776,준대규모점포


## 데이터 전처리

필요한 데이터만 선택

In [86]:
data=data[["소재지전체주소","사업장명","업태구분명"]]
data.head(2)

,소재지전체주소,사업장명,업태구분명
0,"서울특별시 강동구 둔촌동 174-1 B102,B103호",(주)이마트에브리데이 둔촌점,구분없음
1,서울특별시 강동구 둔촌동 170-1,GS THE FRESH 둔촌포레온점,구분없음


## 결측치

In [87]:
data.isna().sum()

소재지전체주소    163
사업장명         0
업태구분명        9
dtype: int64

In [88]:
data=data.dropna(subset=["소재지전체주소"])
data.isna().sum()

소재지전체주소    0
사업장명       0
업태구분명      9
dtype: int64

주소 데이터가 없는건 어쩔 수 없이 없애자

## 주소 분할

In [ ]:
data[["시도", "시군구", "읍면동", "상세주소"]] = data["소재지전체주소"].apply(split_full_address)

In [90]:
data.head()

,소재지전체주소,사업장명,업태구분명,시도,시군구,읍면동,상세주소
0,"서울특별시 강동구 둔촌동 174-1 B102,B103호",(주)이마트에브리데이 둔촌점,구분없음,서울특별시,강동구,둔촌동,"174-1 B102,B103호"
1,서울특별시 강동구 둔촌동 170-1,GS THE FRESH 둔촌포레온점,구분없음,서울특별시,강동구,둔촌동,170-1
2,서울특별시 강동구 둔촌동 170-1,하나로마트 둔촌점,구분없음,서울특별시,강동구,둔촌동,170-1
3,서울특별시 용산구 용산동6가 69-167 신동아쇼핑,신동아쇼핑센터,쇼핑센터,서울특별시,용산구,용산동6가,69-167 신동아쇼핑
4,경상남도 김해시 신문동 506-4,롯데슈퍼 김해신문점,구분없음,경상남도,김해시,신문동,506-4


In [91]:
data.isna().sum()

소재지전체주소     0
사업장명        0
업태구분명       9
시도         12
시군구        12
읍면동        14
상세주소        9
dtype: int64

In [92]:
data[data["시도"].isna()]

,소재지전체주소,사업장명,업태구분명,시도,시군구,읍면동,상세주소
594,1689번지,(주)이마트 대전터미널점,복합쇼핑몰,<NA>,<NA>,<NA>,1689번지
1570,호,부산컴퓨터도매상가,그 밖의 대규모점포,<NA>,<NA>,<NA>,호
1606,호,종로세운상가,대형마트,<NA>,<NA>,<NA>,호
1898,호,화천읍5일장터,그 밖의 대규모점포,<NA>,<NA>,<NA>,호
1911,호,새하나 백화점,백화점,<NA>,<NA>,<NA>,호
2025,호,LG마트춘천점,대형마트,<NA>,<NA>,<NA>,호
2221,호,황전범어타운 상가시장,그 밖의 대규모점포,<NA>,<NA>,<NA>,호
2649,호,사내면5일장터,그 밖의 대규모점포,<NA>,<NA>,<NA>,호
2657,호,사단법인 화천시장조합,그 밖의 대규모점포,<NA>,<NA>,<NA>,호
2778,호,산수시장,시장,<NA>,<NA>,<NA>,호


주소가 이상한건 전부 제거하자

In [93]:
data.dropna(subset=["시도"], inplace=True)
data.isna().sum()

소재지전체주소    0
사업장명       0
업태구분명      9
시도         0
시군구        0
읍면동        2
상세주소       9
dtype: int64

In [94]:
#이후 보기 편하게 정리
data=data[["시도","시군구","읍면동","상세주소","사업장명","업태구분명", "소재지전체주소"]]
data.head()

,시도,시군구,읍면동,상세주소,사업장명,업태구분명,소재지전체주소
0,서울특별시,강동구,둔촌동,"174-1 B102,B103호",(주)이마트에브리데이 둔촌점,구분없음,"서울특별시 강동구 둔촌동 174-1 B102,B103호"
1,서울특별시,강동구,둔촌동,170-1,GS THE FRESH 둔촌포레온점,구분없음,서울특별시 강동구 둔촌동 170-1
2,서울특별시,강동구,둔촌동,170-1,하나로마트 둔촌점,구분없음,서울특별시 강동구 둔촌동 170-1
3,서울특별시,용산구,용산동6가,69-167 신동아쇼핑,신동아쇼핑센터,쇼핑센터,서울특별시 용산구 용산동6가 69-167 신동아쇼핑
4,경상남도,김해시,신문동,506-4,롯데슈퍼 김해신문점,구분없음,경상남도 김해시 신문동 506-4


## 네이버 지도 API 좌표

In [95]:
# 기존 DataFrame의 '전체주소' 컬럼을 리스트로 변환
addresses = data['소재지전체주소'].tolist()

# ThreadPoolExecutor를 이용해 동시 실행
with ThreadPoolExecutor(max_workers=100) as executor:
    results = list(executor.map(get_coordinates_from_address, addresses))

# 결과를 DataFrame으로 변환 후 기존 DataFrame과 결합
df_coordinates = pd.DataFrame(results)
df_final = pd.concat([data.reset_index(drop=True), df_coordinates], axis=1)
print(df_final)

           시도    시군구    읍면동              상세주소                 사업장명  \
0       서울특별시    강동구    둔촌동  174-1 B102,B103호      (주)이마트에브리데이 둔촌점   
1       서울특별시    강동구    둔촌동             170-1  GS THE FRESH 둔촌포레온점   
2       서울특별시    강동구    둔촌동             170-1            하나로마트 둔촌점   
3       서울특별시    용산구  용산동6가      69-167 신동아쇼핑              신동아쇼핑센터   
4        경상남도    김해시    신문동             506-4           롯데슈퍼 김해신문점   
...       ...    ...    ...               ...                  ...   
2787  전북특별자치도    진안군    진안읍      군상리 406번지 7호                 진안시장   
2788     충청남도    천안시    동남구  병천면 병천리 360번지 1호                 병천시장   
2789     경상북도  포항시남구    동해면             도구리 호                 포항시장   
2790     경상북도  포항시남구    동해면             도구리 호                 장기시장   
2791     충청남도    천안시    서북구  성환읍 성환리 449번지 1호                 성환시장   

           업태구분명                           소재지전체주소    Latitude    Longitude  
0           구분없음    서울특별시 강동구 둔촌동 174-1 B102,B103호  37.5215261  127.1368255  
1  

In [96]:
df_final.isna().sum()

시도             0
시군구            0
읍면동            2
상세주소           9
사업장명           0
업태구분명          9
소재지전체주소        0
Latitude     130
Longitude    130
dtype: int64

좌표 130개에 결측치 생성됨

In [97]:
df_final[df_final['Latitude'].isna() | df_final['Longitude'].isna()].head()

,시도,시군구,읍면동,상세주소,사업장명,업태구분명,소재지전체주소,Latitude,Longitude
24,서울특별시,강서구,마곡동,특별계획구역 CP1,LEWEST A관,그 밖의 대규모점포,서울특별시 강서구 마곡동 특별계획구역 CP1,<NA>,<NA>
25,서울특별시,강서구,마곡동,"특별계획구역 CP2, CP3-1, CP3-2",LEWEST B관,그 밖의 대규모점포,"서울특별시 강서구 마곡동 특별계획구역 CP2, CP3-1, CP3-2",<NA>,<NA>
70,인천광역시,계양구,작전동,0 힐스테이트자이계양,GS THE FRESH 인천작전역점,구분없음,인천광역시 계양구 작전동 0 힐스테이트자이계양,<NA>,<NA>
86,인천광역시,미추홀구,주안동,0 포레나 미추홀,아인애비뉴 메디&라이프 복합몰,그 밖의 대규모점포,인천광역시 미추홀구 주안동 0 포레나 미추홀,<NA>,<NA>
94,인천광역시,미추홀구,용현동,0 자이타워,GS THE FRESH 용현자이점,구분없음,인천광역시 미추홀구 용현동 0 자이타워,<NA>,<NA>


이런것들은 주소가 이상한것 같으니 시도 + 시군구 + 읍면동을 합하여 새로 만든 뒤, 자세하진 않아도 대략적인 지역의 좌표를 구해보자

### 좌표 데이터 결측치 전처리

In [98]:
temp=df_final[df_final['Latitude'].isna() | df_final['Longitude'].isna()]
temp.head()

,시도,시군구,읍면동,상세주소,사업장명,업태구분명,소재지전체주소,Latitude,Longitude
24,서울특별시,강서구,마곡동,특별계획구역 CP1,LEWEST A관,그 밖의 대규모점포,서울특별시 강서구 마곡동 특별계획구역 CP1,<NA>,<NA>
25,서울특별시,강서구,마곡동,"특별계획구역 CP2, CP3-1, CP3-2",LEWEST B관,그 밖의 대규모점포,"서울특별시 강서구 마곡동 특별계획구역 CP2, CP3-1, CP3-2",<NA>,<NA>
70,인천광역시,계양구,작전동,0 힐스테이트자이계양,GS THE FRESH 인천작전역점,구분없음,인천광역시 계양구 작전동 0 힐스테이트자이계양,<NA>,<NA>
86,인천광역시,미추홀구,주안동,0 포레나 미추홀,아인애비뉴 메디&라이프 복합몰,그 밖의 대규모점포,인천광역시 미추홀구 주안동 0 포레나 미추홀,<NA>,<NA>
94,인천광역시,미추홀구,용현동,0 자이타워,GS THE FRESH 용현자이점,구분없음,인천광역시 미추홀구 용현동 0 자이타워,<NA>,<NA>


In [99]:
temp["결측치주소"]=data["시도"]+" "+data["시군구"]+" "+data["읍면동"].fillna("")
temp.head(1)

C:\Users\swoo6\AppData\Local\Temp\ipykernel_14132\1199384628.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp["결측치주소"]=data["시도"]+" "+data["시군구"]+" "+data["읍면동"].fillna("")


,시도,시군구,읍면동,상세주소,사업장명,업태구분명,소재지전체주소,Latitude,Longitude,결측치주소
24,서울특별시,강서구,마곡동,특별계획구역 CP1,LEWEST A관,그 밖의 대규모점포,서울특별시 강서구 마곡동 특별계획구역 CP1,<NA>,<NA>,서울특별시 강서구 마곡동


In [100]:
temp.drop(columns=["Latitude","Longitude"], inplace=True, axis=1)
temp.head(1)

C:\Users\swoo6\AppData\Local\Temp\ipykernel_14132\3715193921.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp.drop(columns=["Latitude","Longitude"], inplace=True, axis=1)


,시도,시군구,읍면동,상세주소,사업장명,업태구분명,소재지전체주소,결측치주소
24,서울특별시,강서구,마곡동,특별계획구역 CP1,LEWEST A관,그 밖의 대규모점포,서울특별시 강서구 마곡동 특별계획구역 CP1,서울특별시 강서구 마곡동


In [101]:
# 기존 DataFrame의 '전체주소' 컬럼을 리스트로 변환
addresses = temp['결측치주소'].tolist()

# ThreadPoolExecutor를 이용해 동시 실행
with ThreadPoolExecutor(max_workers=100) as executor:
    results = list(executor.map(get_coordinates_from_address, addresses))

# 결과를 DataFrame으로 변환 후 기존 DataFrame과 결합
df_coordinates = pd.DataFrame(results)
temp_final = pd.concat([temp.reset_index(drop=True), df_coordinates], axis=1)
print(temp_final)

        시도    시군구  읍면동                      상세주소                 사업장명  \
0    서울특별시    강서구  마곡동                특별계획구역 CP1            LEWEST A관   
1    서울특별시    강서구  마곡동  특별계획구역 CP2, CP3-1, CP3-2            LEWEST B관   
2    인천광역시    계양구  작전동               0 힐스테이트자이계양  GS THE FRESH 인천작전역점   
3    인천광역시   미추홀구  주안동                 0 포레나 미추홀     아인애비뉴 메디&라이프 복합몰   
4    인천광역시   미추홀구  용현동                    0 자이타워   GS THE FRESH 용현자이점   
..     ...    ...  ...                       ...                  ...   
125   경상북도  포항시북구  송라면                     광천리 호                 송라시장   
126   경상북도  포항시북구  송라면                     광천리 호                 기계시장   
127   경상북도  포항시남구  동해면                     도구리 호                 죽장시장   
128   경상북도  포항시남구  동해면                     도구리 호                 포항시장   
129   경상북도  포항시남구  동해면                     도구리 호                 장기시장   

          업태구분명                                 소재지전체주소           결측치주소  \
0    그 밖의 대규모점포                서울특별시 강서구 마곡동 특별계

In [103]:
temp_final.isna().sum()

시도           0
시군구          0
읍면동          2
상세주소         2
사업장명         0
업태구분명        1
소재지전체주소      0
결측치주소        2
Latitude     3
Longitude    3
dtype: int64

결측치 세개는 구글 지도를 통해 넣자

In [104]:
temp_final[temp_final['Latitude'].isna() | temp_final['Longitude'].isna()]

,시도,시군구,읍면동,상세주소,사업장명,업태구분명,소재지전체주소,결측치주소,Latitude,Longitude
38,대구광역시,달서구,월성1동,89번지 2호,홈플러스(주)익스프레스 월성점,대형마트,대구광역시 달서구 월성1동 89번지 2호,NaN,<NA>,<NA>
46,경기도,파주시,문산읍,당동리 산업단지내 지원시설용지 1-1,홈플러스(주)파주문산점,대형마트,경기도 파주시 문산읍 당동리 산업단지내 지원시설용지 1-1,경기도 파주시 교하읍,<NA>,<NA>
79,경상남도,거제시,신현읍,고현리 961번지 59호,회원프라자,그 밖의 대규모점포,경상남도 거제시 신현읍 고현리 961번지 59호,NaN,<NA>,<NA>


In [106]:
temp_final.loc[38, ["Latitude", "Longitude"]]=[35.824818, 128.526019]
temp_final.loc[46, ["Latitude", "Longitude"]]=[37.862314, 126.779992]
temp_final.loc[79, ["Latitude", "Longitude"]]=[34.888525, 128.620431]
temp_final.loc[[38,46,79]]

,시도,시군구,읍면동,상세주소,사업장명,업태구분명,소재지전체주소,결측치주소,Latitude,Longitude
38,대구광역시,달서구,월성1동,89번지 2호,홈플러스(주)익스프레스 월성점,대형마트,대구광역시 달서구 월성1동 89번지 2호,NaN,35.824818,128.526019
46,경기도,파주시,문산읍,당동리 산업단지내 지원시설용지 1-1,홈플러스(주)파주문산점,대형마트,경기도 파주시 문산읍 당동리 산업단지내 지원시설용지 1-1,경기도 파주시 교하읍,37.862314,126.779992
79,경상남도,거제시,신현읍,고현리 961번지 59호,회원프라자,그 밖의 대규모점포,경상남도 거제시 신현읍 고현리 961번지 59호,NaN,34.888525,128.620431


In [108]:
temp_final.isna().sum()

시도           0
시군구          0
읍면동          2
상세주소         2
사업장명         0
업태구분명        1
소재지전체주소      0
결측치주소        2
Latitude     0
Longitude    0
dtype: int64

### df_final의 좌표 결측치를 temp_final의 좌표로 채우기

In [109]:
df_final.head()

,시도,시군구,읍면동,상세주소,사업장명,업태구분명,소재지전체주소,Latitude,Longitude
0,서울특별시,강동구,둔촌동,"174-1 B102,B103호",(주)이마트에브리데이 둔촌점,구분없음,"서울특별시 강동구 둔촌동 174-1 B102,B103호",37.5215261,127.1368255
1,서울특별시,강동구,둔촌동,170-1,GS THE FRESH 둔촌포레온점,구분없음,서울특별시 강동구 둔촌동 170-1,37.5269345,127.1387574
2,서울특별시,강동구,둔촌동,170-1,하나로마트 둔촌점,구분없음,서울특별시 강동구 둔촌동 170-1,37.5269345,127.1387574
3,서울특별시,용산구,용산동6가,69-167 신동아쇼핑,신동아쇼핑센터,쇼핑센터,서울특별시 용산구 용산동6가 69-167 신동아쇼핑,37.5172474,126.9852079
4,경상남도,김해시,신문동,506-4,롯데슈퍼 김해신문점,구분없음,경상남도 김해시 신문동 506-4,35.1955769,128.8150481


In [111]:
temp_1 = temp_final[['소재지전체주소', 'Latitude', 'Longitude']]
df_merged = df_final.merge(temp_1, on='소재지전체주소', how='left', suffixes=('', '_temp'))
df_merged.head(1)

,시도,시군구,읍면동,상세주소,사업장명,업태구분명,소재지전체주소,Latitude,Longitude,Latitude_temp,Longitude_temp
0,서울특별시,강동구,둔촌동,"174-1 B102,B103호",(주)이마트에브리데이 둔촌점,구분없음,"서울특별시 강동구 둔촌동 174-1 B102,B103호",37.5215261,127.1368255,NaN,NaN


In [112]:
df_merged['Latitude'] = df_merged['Latitude'].fillna(df_merged['Latitude_temp'])
df_merged['Longitude'] = df_merged['Longitude'].fillna(df_merged['Longitude_temp'])

In [113]:
df_merged.isna().sum()

시도                   0
시군구                  0
읍면동                  4
상세주소                11
사업장명                 0
업태구분명                9
소재지전체주소              0
Latitude             0
Longitude            0
Latitude_temp     2662
Longitude_temp    2662
dtype: int64

좌표를 무사히 다 채워넣었다.

## 마무리 전처리

In [114]:
df_merged=df_merged[["시도","시군구","읍면동","상세주소","사업장명","업태구분명","Latitude","Longitude"]]

In [115]:
df_merged.columns=["County","City","Town","Address","Store_Name","Store_Type","Latitude","Longitude"]
df_merged.head(1)

,County,City,Town,Address,Store_Name,Store_Type,Latitude,Longitude
0,서울특별시,강동구,둔촌동,"174-1 B102,B103호",(주)이마트에브리데이 둔촌점,구분없음,37.5215261,127.1368255


In [118]:
df_merged.County.unique()

array(['서울특별시', '경상남도', '대구광역시', '인천광역시', '부산광역시', '충청남도', '세종특별자치시',
       '경기도', '강원특별자치도', '경상북도', '충청북도', '전북특별자치도', '대전광역시', '울산광역시',
       '전라남도', '제주특별자치도', '광주광역시', '133번지'], dtype=object)

113번지가 주소(시도)에 잘못 들어가 있다

In [121]:
df_merged.query("County=='133번지'")

,County,City,Town,Address,Store_Name,Store_Type,Latitude,Longitude
704,133번지,1호,<NA>,<NA>,(주)이마트 봉담점,대형마트,37.528317,126.929425
705,133번지,1호,<NA>,<NA>,(주)이마트 봉담점,대형마트,35.169201,128.164238
707,133번지,1호,<NA>,<NA>,이마트 화성봉담점,대형마트,37.528317,126.929425
708,133번지,1호,<NA>,<NA>,이마트 화성봉담점,대형마트,35.169201,128.164238


In [124]:
# 그냥 전부 제거하자
df_merged=df_merged.query("County!='133번지'")

## 저장

In [ ]:
df_merged.to_csv("결과데이터/전국시장정보.csv", index=False, encoding="cp949")

# 4. 전국 영화관 정보

In [ ]:
data=pd.read_excel("원본데이터/03_13_02_P_영화상영관.xlsx")
data=data[["소재지전체주소","사업장명","공연장형태구분명"]] #필요한 데이터만
data.head(1)

c:\ProgramData\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,소재지전체주소,사업장명,공연장형태구분명
0,전라남도 강진군 강진읍 동성리 192-9 강진군 어울림센터 3층,강진영화관 1관,영화관


In [128]:
data.isna().sum()

소재지전체주소      0
사업장명         0
공연장형태구분명    39
dtype: int64

공연장형태구분명은 중요한게 아니니 넘어가자

## 영화관 중복 처리

같은 건물에 영화관 이름이 같은게 많다

In [129]:
data=data.drop_duplicates("소재지전체주소")

## 주소 분할

In [ ]:
data[["시도", "시군구", "읍면동", "상세주소"]] = data["소재지전체주소"].apply(split_full_address)

KeyError: '소재지전체주소'

In [131]:
data.isna().sum()

소재지전체주소     0
사업장명        0
공연장형태구분명    9
시도          0
시군구         0
읍면동         0
상세주소        1
dtype: int64

In [133]:
data.시도.unique()

array(['전라남도', '인천광역시', '경상북도', '서울특별시', '충청북도', '강원특별자치도', '부산광역시',
       '광주광역시', '경기도', '울산광역시', '전북특별자치도', '대구광역시', '경상남도', '제주특별자치도',
       '대전광역시', '충청남도', '세종특별자치시', '전라북도'], dtype=object)

전북특별자치도와 전라북도를 하나로 통합하자

In [137]:
data.replace({"시도": {"전북특별자치도": "전라북도"}}, inplace=True)
data.시도.unique()

array(['전라남도', '인천광역시', '경상북도', '서울특별시', '충청북도', '강원특별자치도', '부산광역시',
       '광주광역시', '경기도', '울산광역시', '전라북도', '대구광역시', '경상남도', '제주특별자치도',
       '대전광역시', '충청남도', '세종특별자치시'], dtype=object)

## 네이버 지도 API 좌표

In [139]:
# 기존 DataFrame의 '전체주소' 컬럼을 리스트로 변환
addresses = data['소재지전체주소'].tolist()

# ThreadPoolExecutor를 이용해 동시 실행
with ThreadPoolExecutor(max_workers=100) as executor:
    results = list(executor.map(get_coordinates_from_address, addresses))

# 결과를 DataFrame으로 변환 후 기존 DataFrame과 결합
df_coordinates = pd.DataFrame(results)
df_final = pd.concat([data.reset_index(drop=True), df_coordinates], axis=1)
print(df_final)

                                  소재지전체주소                   사업장명 공연장형태구분명  \
0    전라남도 강진군 강진읍 동성리 192-9 강진군 어울림센터 3층                강진영화관 1관      영화관   
1             인천광역시 미추홀구 학익동 0 시티오씨엘 3단지   메가박스중앙(주)인천학익지점 컴포트5관      영화관   
2         경상북도 의성군 안계면 용기리 843-5 안계행복플랫폼                 안계행복영화관      영화관   
3      경상북도 의성군 의성읍 후죽리 508-1 의성군청소년문화의집              의성작은영화관 2관      영화관   
4              서울특별시 마포구 서교동 354-5 라이즈호텔             모노플렉스 앳 라이즈      영화관   
..                                    ...                    ...      ...   
798                 경기도 동두천시 생연동 616-5번지                 문화극장 2관      영화관   
799                   경기도 동두천시 생연동 616-2                 문화극장 1관      영화관   
800                강원특별자치도 원주시 학성동 329-2                    문화극장      영화관   
801                  강원특별자치도 원주시 평원동 368                    아카데미      영화관   
802                 경기도 동두천시 생연동 828-2번지                 문화극장 3관      영화관   

          시도   시군구  읍면동                    상세주소    Latitude    Longitude  


In [140]:
df_final.isna().sum()

소재지전체주소       0
사업장명          0
공연장형태구분명      9
시도            0
시군구           0
읍면동           0
상세주소          1
Latitude     10
Longitude    10
dtype: int64

좌표 결측치 10개는 직접 입력하자

In [141]:
df_final[df_final['Latitude'].isna() | df_final['Longitude'].isna()]

,소재지전체주소,사업장명,공연장형태구분명,시도,시군구,읍면동,상세주소,Latitude,Longitude
1,인천광역시 미추홀구 학익동 0 시티오씨엘 3단지,메가박스중앙(주)인천학익지점 컴포트5관,영화관,인천광역시,미추홀구,학익동,0 시티오씨엘 3단지,<NA>,<NA>
29,제주특별자치도 서귀포시 안덕면 서광리 지하1층,씨네라운지 2관,영화관,제주특별자치도,서귀포시,안덕면,서광리 지하1층,<NA>,<NA>
37,인천광역시 서구 가정동 0 루원지웰시티푸르지오 상가동 지하1호,CGV 인천가정 제3관,영화관,인천광역시,서구,가정동,0 루원지웰시티푸르지오 상가동 지하1호,<NA>,<NA>
43,인천광역시 미추홀구 주안동 0 포레나 미추홀,CGV인천시민공원 1관,영화관,인천광역시,미추홀구,주안동,0 포레나 미추홀,<NA>,<NA>
255,경기도 성남시 수정구 창곡동 23-4 위례한화오벨리스크센트럴스퀘어 지하1층,롯데컬처웍스(주) 롯데시네마위례 6관,영화관,경기도,성남시,수정구,창곡동 23-4 위례한화오벨리스크센트럴스퀘어 지하1층,<NA>,<NA>
320,경기도 의정부시 민락동 메가타워 메가박스 5~8층,메가박스 의정부 민락점 3관,영화관,경기도,의정부시,민락동,메가타워 메가박스 5~8층,<NA>,<NA>
360,전라남도 여수시 웅천동 1699-501,CGV여수웅천(제1관),영화관,전라남도,여수시,웅천동,1699-501,<NA>,<NA>
681,경기도 광명시 하안동 1번지,미래상영관(이동상영관),영화관,경기도,광명시,하안동,1번지,<NA>,<NA>
770,광주광역시 남구 주월동 126620번지,호남시청각사,영화관,광주광역시,남구,주월동,126620번지,<NA>,<NA>
801,강원특별자치도 원주시 평원동 368,아카데미,영화관,강원특별자치도,원주시,평원동,368,<NA>,<NA>


In [142]:
#숫자형으로 전환
df_final[["Latitude","Longitude"]]=df_final[["Latitude","Longitude"]].apply(pd.to_numeric, errors='coerce')

In [143]:
df_final.loc[1, ["Latitude", "Longitude"]]=[37.441007, 126.647164]
df_final.loc[29, ["Latitude", "Longitude"]]=[33.297567, 126.293790]
df_final.loc[37, ["Latitude", "Longitude"]]=[37.526592, 126.669564]
df_final.loc[43, ["Latitude", "Longitude"]]=[37.457447, 126.681004]
df_final.loc[255, ["Latitude", "Longitude"]]=[37.472478, 127.140793]
df_final.loc[320, ["Latitude", "Longitude"]]=[37.745748, 127.095728]
df_final.loc[360, ["Latitude", "Longitude"]]=[34.746502, 127.671619]
df_final.loc[681, ["Latitude", "Longitude"]]=[37.460205, 126.869050]
df_final.loc[770, ["Latitude", "Longitude"]]=[35.132050, 126.900308]
df_final.loc[801, ["Latitude", "Longitude"]]=[37.350957, 127.952082]

In [144]:
df_final.isna().sum()

소재지전체주소      0
사업장명         0
공연장형태구분명     9
시도           0
시군구          0
읍면동          0
상세주소         1
Latitude     0
Longitude    0
dtype: int64

## 전처리 마무리

In [149]:
df_final=df_final[["시도","시군구","읍면동","상세주소","사업장명","공연장형태구분명","Latitude","Longitude"]]
df_final.columns=["County","City","Town","Address","Theater_Name","Theater_Type","Latitude","Longitude"]

In [150]:
df_final.head(1)

,County,City,Town,Address,Theater_Name,Theater_Type,Latitude,Longitude
0,전라남도,강진군,강진읍,동성리 192-9 강진군 어울림센터 3층,강진영화관 1관,영화관,34.640764,126.772709


In [ ]:
df_final.to_csv("결과데이터/전국영화관정보.csv", index=False, encoding="cp949")

# 5. 전국 병원 정보

In [8]:
data=pd.read_excel("원본데이터/01_01_01_P_병원.xlsx")
data=data[["소재지전체주소","사업장명","의료기관종별명"]]
data.head(1)

c:\ProgramData\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,소재지전체주소,사업장명,의료기관종별명
0,인천광역시 서구 가정동 0 8~10층,바로서구병원,병원


In [9]:
data.isna().sum()

소재지전체주소    1403
사업장명          0
의료기관종별명     523
dtype: int64

주소가 없는거는 없애자

In [10]:
data.dropna(subset=["소재지전체주소"], inplace=True)
data.isna().sum()

소재지전체주소      0
사업장명         0
의료기관종별명    523
dtype: int64

In [11]:
data[["사업장명","소재지전체주소"]].value_counts()

사업장명                소재지전체주소                            
지성재활요양병원            제주특별자치도 제주시 애월읍 광령리 100번지 1호 1호        2
의료법인 의명의료재단 마음편한병원                                         2
강남한방병원                                                     2
그랜드치과병원                                                    2
희망병원                                                       2
                                                          ..
상남굿모닝내과병원           창원시 상남동 73-3 서울메디칼빌딩 6,7층              1
상동서울한방병원            경기도 부천시 원미구 상동 547-1 코오롱이데아폴리스 2층일부    1
상록병원                광주광역시 광산구 우산동  1580번지 3호 3호            1
상록수요양병원             울산광역시 동구 전하동 653-2 상록수요양병원             1
老世노세요양병원            광주광역시 광산구 송정동  1364번지                  1
Name: count, Length: 4099, dtype: int64

같은 건물에 병원 이름이 다른게 많다

In [12]:
data=data.drop_duplicates(["소재지전체주소","사업장명"])
data.head()

,소재지전체주소,사업장명,의료기관종별명
0,인천광역시 서구 가정동 0 8~10층,바로서구병원,병원
1,경기도 용인시 처인구 모현읍 능원리 107-1,승일희망요양병원,요양병원(일반요양병원)
2,서울특별시 종로구 원남동 177-1,포레스트병원,병원
3,대전광역시 동구 신안동 293번지 1호,대전화병원,병원
4,인천광역시 연수구 옥련동 186 명신빌딩,명신더블유한방병원,한방병원


## 주소 분할

In [13]:
data[["시도", "시군구", "읍면동", "상세주소"]] = data["소재지전체주소"].apply(split_full_address)

In [14]:
data.시도.unique()

array(['인천광역시', '경기도', '서울특별시', '대전광역시', '경상남도', '울산광역시', '전라남도', '충청북도',
       '경상북도', '강원특별자치도', '대구광역시', '부산광역시', '세종특별자치시', <NA>, '광주광역시',
       '충청남도', '전북특별자치도', '제주특별자치도', '123번지', '제천시', '연향동', '인지동',
       '부산광역시해운대구', '죽산면', '대룡동', '조례동', '금광면', '포항시', '경북', '인월동', '창원시',
       '예산읍', '159번지', '587번지', '해룡면', '시흥시', '안동시', '온천3동', '1517번지',
       '청주시상당구', '진주시', '고성군', '매곡동', '안락1동', '장천동', '영주시', '명륜1동'],
      dtype=object)

### 주소에 오류가 있는 것들을 확인

살릴 수 있는것만 살리자

In [15]:
data.dropna(subset=["시도"], inplace=True)

1. 시도(맨 앞)를 수정
2. 이를 전체 주소로 합친다
3. 이를 다시 분할한다

In [16]:
#'동'은 중복 지명이 많을 경우 제한다
data.replace({"시도": {"제천시": "충청북도 제천시"}}, inplace=True)
data.replace({"시도": {"부산광역시해운대구": "부산광역시 해운대구"}}, inplace=True)
data.replace({"시도": {"죽산면": "경기도 안성시 죽산면"}}, inplace=True)
data.replace({"시도": {"조례동": "전라남도 순천시 조례동"}}, inplace=True)
data.replace({"시도": {"금광면": "경기도 안성시 금광면"}}, inplace=True)
data.replace({"시도": {"포항시": "경상북도 포항시"}}, inplace=True)
data.replace({"시도": {"경북": "경상북도"}}, inplace=True)
data.replace({"시도": {"인월동": "전라남도 순천시 인월동"}}, inplace=True)
data.replace({"시도": {"창원시": "경상남도 창원시"}}, inplace=True)
data.replace({"시도": {"예산읍": "충청남도 예산군 예산읍"}}, inplace=True)
data.replace({"시도": {"해룡면": "전라남도 순천시 해룡면"}}, inplace=True)
data.replace({"시도": {"시흥시": "경기도 시흥시"}}, inplace=True)
data.replace({"시도": {"안동시": "경상북도 안동시"}}, inplace=True)
data.replace({"시도": {"온천3동": "부산광역시 동래구 온천3동"}}, inplace=True)
data.replace({"시도": {"청주시상당구": "충청북도 청주시 상당구"}}, inplace=True)
data.replace({"시도": {"진주시": "경상남도 진주시"}}, inplace=True)
data.replace({"시도": {"안락1동": "부산광역시 동래구 안락1동"}}, inplace=True)
data.replace({"시도": {"영주시": "경상북도 영주시"}}, inplace=True)
data.replace({"시도": {"명륜1동": "강원특별자치도 원주시 명륜1동"}}, inplace=True)

주소 합치고 다시 분할

In [17]:
data["전체주소"]=data["시도"]+" "+data["시군구"]+" "+data["읍면동"].fillna("")+" "+data["상세주소"].fillna("")
data[["시도", "시군구", "읍면동", "상세주소"]] = data["전체주소"].apply(split_full_address)

In [18]:
data.시도.unique()

array(['인천광역시', '경기도', '서울특별시', '대전광역시', '경상남도', '울산광역시', '전라남도', '충청북도',
       '경상북도', '강원특별자치도', '대구광역시', '부산광역시', '세종특별자치시', '광주광역시', '충청남도',
       '전북특별자치도', '제주특별자치도', '123번지', '연향동', '인지동', '대룡동', '159번지',
       '587번지', '1517번지', '고성군', '매곡동', '장천동'], dtype=object)

In [19]:
remove_values = [
    '123번지', '연향동', '인지동', '대룡동', '159번지',
    '587번지', '1517번지', '고성군', '매곡동', '장천동'
]
data = data[~data['시도'].isin(remove_values)]

In [20]:
data.isna().sum()

소재지전체주소      0
사업장명         0
의료기관종별명    204
시도           0
시군구          0
읍면동          1
상세주소        44
전체주소         0
dtype: int64

주소 전처리 완료

In [21]:
data=data[["시도","시군구","읍면동","상세주소","사업장명","의료기관종별명", "전체주소"]]
data.head(1)

,시도,시군구,읍면동,상세주소,사업장명,의료기관종별명,전체주소
0,인천광역시,서구,가정동,0 8~10층,바로서구병원,병원,인천광역시 서구 가정동 0 8~10층


## 네이버 지도 API 좌표

In [22]:
# 기존 DataFrame의 '전체주소' 컬럼을 리스트로 변환
addresses = data['전체주소'].tolist()

# ThreadPoolExecutor를 이용해 동시 실행
with ThreadPoolExecutor(max_workers=100) as executor:
    results = list(executor.map(get_coordinates_from_address, addresses))

# 결과를 DataFrame으로 변환 후 기존 DataFrame과 결합
df_coordinates = pd.DataFrame(results)
df_final = pd.concat([data.reset_index(drop=True), df_coordinates], axis=1)
print(df_final)

           시도  시군구   읍면동                     상세주소                 사업장명  \
0       인천광역시   서구   가정동                  0 8~10층               바로서구병원   
1         경기도  용인시   처인구            모현읍 능원리 107-1             승일희망요양병원   
2       서울특별시  종로구   원남동                    177-1               포레스트병원   
3       대전광역시   동구   신안동                 293번지 1호                대전화병원   
4       인천광역시  연수구   옥련동                 186 명신빌딩            명신더블유한방병원   
...       ...  ...   ...                      ...                  ...   
3374    대전광역시  대덕구   덕암동                     43-5  의료법인원전의료재단보니파시오요양병원   
3375     경상북도  청도군   화양읍                범곡리 101-5           군립청도노인요양병원   
3376    인천광역시   서구  석남1동      522번지 1호 석남1동 523-1      (의)성세의료재단 뉴성민병원   
3377    인천광역시   서구   연희동  793번지 1호 우성메디피아 2층,4층일부               청라국제병원   
3378  강원특별자치도  속초시   조양동                1013번지 1호              속초정요양병원   

           의료기관종별명                                  전체주소    Latitude  \
0               병원                  인천광

In [23]:
df_final.isna().sum()

시도             0
시군구            0
읍면동            1
상세주소          44
사업장명           0
의료기관종별명      204
전체주소           0
Latitude      69
Longitude     69
dtype: int64

좌표가 없는 것은 그냥 지나가겠다

In [24]:
df_final.drop(columns=["전체주소"], axis=1, inplace=True)

In [25]:
df_final.head(1)

,시도,시군구,읍면동,상세주소,사업장명,의료기관종별명,Latitude,Longitude
0,인천광역시,서구,가정동,0 8~10층,바로서구병원,병원,<NA>,<NA>


In [26]:
df_final[["Latitude","Longitude"]]=df_final[["Latitude","Longitude"]].apply(pd.to_numeric, errors='coerce')

## 전처리 마무리

In [27]:
df_final.columns=["County","City","Town","Address","Hospital_Name","Hospital_Type","Latitude","Longitude"]
df_final.head(1)

,County,City,Town,Address,Hospital_Name,Hospital_Type,Latitude,Longitude
0,인천광역시,서구,가정동,0 8~10층,바로서구병원,병원,NaN,NaN


In [ ]:
df_final.to_csv("결과데이터/전국병원정보.csv", index=False, encoding="cp949")

# 6. 전국 학교 정보

전국 초,중,고교의 정보

In [29]:
data=pd.read_csv("원본데이터/전국초중등학교위치표준데이터.csv", encoding="cp949")

In [220]:
data=data[["학교명","학교급구분","소재지지번주소", "위도","경도"]]
data.head(1)

,학교명,학교급구분,소재지지번주소,위도,경도
0,대전용산초등학교,초등학교,대전광역시 유성구 용산동 662,36.420154,127.395076


In [221]:
data.isna().sum()

학교명        0
학교급구분      0
소재지지번주소    0
위도         0
경도         0
dtype: int64

## 주소 분할

In [222]:
data[["시도", "시군구", "읍면동", "상세주소"]] = data["소재지지번주소"].apply(split_full_address)

In [223]:
data.시도.unique()

array(['대전광역시', '부산광역시', '서울특별시', '제주특별자치도', '충청남도', '경기도', '광주광역시',
       '세종특별자치시', '전북특별자치도', '울산광역시', '강원특별자치도', '충청북도', '대구광역시', '경상북도',
       '인천광역시', '경상남도', '전라남도', '김해시'], dtype=object)

김해시 하나만 시도 주소에 잘못 들어갔다

In [224]:
data.query("시도=='김해시'")

,학교명,학교급구분,소재지지번주소,위도,경도,시도,시군구,읍면동,상세주소
8256,김해서중학교,중학교,김해시 주촌면 선지리 1468,35.233352,128.846022,김해시,주촌면,선지리,1468


In [230]:
data.loc[8256, "소재지지번주소"]="경상남도 김해시 주촌면 선지리 1468"

다시 주소 분할

In [233]:
data.drop(columns=["시도","시군구","읍면동","상세주소"], axis=1, inplace=True)
data[["시도", "시군구", "읍면동", "상세주소"]] = data["소재지지번주소"].apply(split_full_address)

In [234]:
data.시도.unique()

array(['대전광역시', '부산광역시', '서울특별시', '제주특별자치도', '충청남도', '경기도', '광주광역시',
       '세종특별자치시', '전북특별자치도', '울산광역시', '강원특별자치도', '충청북도', '대구광역시', '경상북도',
       '인천광역시', '경상남도', '전라남도'], dtype=object)

In [236]:
data=data[["학교명","학교급구분","시도", "시군구", "읍면동", "상세주소", "위도","경도"]]
data.columns=["School_Name","School_Type","County","City","Town","Address","Latitude","Longitude"]
data.head(1)

,School_Name,School_Type,County,City,Town,Address,Latitude,Longitude
0,대전용산초등학교,초등학교,대전광역시,유성구,용산동,662,36.420154,127.395076


## 초, 중, 고교 분리하기

In [237]:
data_element_school=data[data['School_Type']=='초등학교']
data_middle_school=data[data['School_Type']=='중학교']
data_high_school=data[data['School_Type']=='고등학교']
data_element_school.head()

,School_Name,School_Type,County,City,Town,Address,Latitude,Longitude
0,대전용산초등학교,초등학교,대전광역시,유성구,용산동,662,36.420154,127.395076
1,상당초등학교,초등학교,부산광역시,해운대구,좌동,1405,35.179656,129.176031
4,물메초등학교,초등학교,제주특별자치도,제주시,애월읍,수산리 1033,33.466188,126.391432
7,마곡초등학교,초등학교,충청남도,공주시,사곡면,운암리 459,36.558955,127.017743
10,부당초등학교,초등학교,전북특별자치도,무주군,부남면,가당리 448-2,36.015598,127.580283


## 각각 저장하기

In [ ]:
data_element_school.to_csv("결과데이터/전국초등학교정보.csv", index=False, encoding="cp949")
data_middle_school.to_csv("결과데이터/전국중등학교정보.csv", index=False, encoding="cp949")
data_high_school.to_csv("결과데이터/전국고등학교정보.csv", index=False, encoding="cp949")

# 7. 전국 공장 정보

원본데이터는 엑셀이나 어떤 옵션으로도 불러오지 못해서 csv파일로 따로 저장

In [ ]:
data=pd.read_csv("원본데이터/(2025.02월말기준)_전국공장등록현황.csv", encoding="utf-8-sig")
data=data[["회사명","공장규모","업종명","공장주소_지번"]]
data.head(1)

C:\Users\swoo6\AppData\Local\Temp\ipykernel_14132\1398049239.py:1: DtypeWarning: Columns (10,12,29) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv("(2025.02월말기준)_전국공장등록현황.csv", encoding="utf-8-sig")


,회사명,공장규모,업종명,공장주소_지번
0,남일문화 (주),소기업,경 인쇄업 외 2 종,서울특별시 종로구 창성동 87-1번지


In [240]:
data.isna().sum()

회사명        0
공장규모       0
업종명        0
공장주소_지번    0
dtype: int64

## 주소 분할

In [242]:
data[["시도", "시군구", "읍면동", "상세주소"]] = data["공장주소_지번"].apply(split_full_address)

In [244]:
data.isna().sum()

회사명          0
공장규모         0
업종명          0
공장주소_지번      0
시도          37
시군구         37
읍면동        113
상세주소       743
dtype: int64

In [245]:
data.dropna(subset=["시도"], inplace=True)
data.isna().sum()

회사명          0
공장규모         0
업종명          0
공장주소_지번      0
시도           0
시군구          0
읍면동         76
상세주소       743
dtype: int64

In [246]:
data.시도.unique()

array(['서울특별시', '충무로5가', '인천광역시', '구로동', '고척동', '온수동', '가산동', '부산광역시',
       '경상남도', '동삼동', '부산광역시동래구', '용당동', '부산광역시해운대구', '하단동', '장림동', '회동동',
       '광주광역시', '부산광역시강서구', '미음동', '연산동', '광안동', '부산광역시사상구', '감전동', '학장동',
       '장안읍', '부산광역시기장군', '대구광역시', '지저동', '비산동', '이현동', '노원동3가', '검단동',
       '호림동', '월성동', '호산동', '논공읍', '유가읍', '옥포읍', '도화동', '주안동', '논현동',
       '고잔동', '인천광역시남동구', '십정동', '청천동', '갈산동', '서운동', '인천광역시서구', '가좌동',
       '오류동', '청라동', '두암동', '광주광역시북구', '지야동', '소촌동', '도천동', '진곡동', '오선동',
       '대전광역시', '낭월동', '장대동', '대전광역시유성구', '탑립동', '신동', '오정동', '대화동',
       '신일동', '울산광역시', '황성동', '달천동', '청량읍', '세종특별자치시', '집현동', '충청남도',
       '연동면', '세종특별자치시부강면', '경기도', '시흥동', '상대원동', '안양동', '춘의동', '도당동',
       '역곡동', '삼정동', '내동', '세교동', '청북읍', '진위면', '지행동', '경기도안산시', '팔곡이동',
       '성곡동', '원시동', '목내동', '갈매동', '인창동', '오남읍', '양산동', '은행동', '매화동',
       '도창동', '정왕동', '배곧동', '당동', '경기도군포시', '당정동', '산본동', '삼동', '경기도의왕시',
       '풍산동', '이동읍', '양지면', '신갈동', '영덕동', '풍덕천동', '검산동', '신촌동', 

오류가 있는 주소가 많다. 다 수정해보자

In [248]:
data.replace({"시도": {"충무로5가": "서울특별시 중구 충무로5가"}}, inplace=True)
data.replace({"시도": {"구로동": "서울특별시 구로구 구로동"}}, inplace=True)
data.replace({"시도": {"고척동": "서울특별시 구로구 고척동"}}, inplace=True)
data.replace({"시도": {"온수동": "서울특별시 구로구 온수동"}}, inplace=True)
data.replace({"시도": {"가산동": "서울특별시 금천구 가산동"}}, inplace=True)
data.replace({"시도": {"동삼동": "부산광역시 영도구 동삼동"}}, inplace=True)
data.replace({"시도": {"부산광역시동래구": "부산광역시 동래구"}}, inplace=True)
data.replace({"시도": {"부산광역시해운대구": "부산광역시 해운대구"}}, inplace=True)
data.replace({"시도": {"하단동": "부산광역시 사하구 하단동"}}, inplace=True)
data.replace({"시도": {"장림동": "부산광역시 사하구 장림동"}}, inplace=True)
data.replace({"시도": {"회동동": "부산광역시 금정구 회동동"}}, inplace=True)
data.replace({"시도": {"부산광역시강서구": "부산광역시 강서구"}}, inplace=True)
data.replace({"시도": {"미음동": "부산광역시 강서구 미음동"}}, inplace=True)
data.replace({"시도": {"광안동": "부산광역시 수영구 광안동"}}, inplace=True)
data.replace({"시도": {"부산광역시사상구": "부산광역시 사상구"}}, inplace=True)
data.replace({"시도": {"감전동": "부산광역시 사상구 감전동"}}, inplace=True)
data.replace({"시도": {"학장동": "부산광역시 사상구 학장동"}}, inplace=True)
data.replace({"시도": {"장안읍": "부산광역시 기장군 장안읍"}}, inplace=True)
data.replace({"시도": {"부산광역시기장군": "부산광역시 기장군"}}, inplace=True)
data.replace({"시도": {"지저동": "대구광역시 동구 지저동"}}, inplace=True)
data.replace({"시도": {"노원동3가": "대구광역시 북구 노원동3가"}}, inplace=True)
data.replace({"시도": {"검단동": "대구광역시 북구 검단동"}}, inplace=True)
data.replace({"시도": {"호림동": "대구광역시 달서구 호림동"}}, inplace=True)
data.replace({"시도": {"월성동": "대구광역시 달서구 월성동"}}, inplace=True)
data.replace({"시도": {"호산동": "대구광역시 달서구 호산동"}}, inplace=True)
data.replace({"시도": {"논공읍": "대구광역시 달성군 논공읍"}}, inplace=True)
data.replace({"시도": {"유가읍": "대구광역시 달성군 유가읍"}}, inplace=True)

In [249]:
data.replace({"시도": {"옥포읍": "대구광역시 달성군 옥포읍"}}, inplace=True)
data.replace({"시도": {"도화동": "인천광역시 미추홀구 도화동"}}, inplace=True)
data.replace({"시도": {"주안동": "인천광역시 미추홀구 주안동"}}, inplace=True)
data.replace({"시도": {"논현동": "인천광역시 남동구 논현동"}}, inplace=True)
data.replace({"시도": {"고잔동": "인천광역시 남동구 고잔동"}}, inplace=True)
data.replace({"시도": {"인천광역시남동구": "인천광역시 남동구"}}, inplace=True)
data.replace({"시도": {"십정동": "인천광역시 부평구 십정동"}}, inplace=True)
data.replace({"시도": {"청천동": "인천광역시 부평구 청천동"}}, inplace=True)
data.replace({"시도": {"갈산동": "인천광역시 부평구 갈산동"}}, inplace=True)
data.replace({"시도": {"서운동": "인천광역시 계양구 서운동"}}, inplace=True)
data.replace({"시도": {"인천광역시서구": "인천광역시 서구"}}, inplace=True)
data.replace({"시도": {"가좌동": "인천광역시 서구 가좌동"}}, inplace=True)
data.replace({"시도": {"오류동": "인천광역시 서구 오류동"}}, inplace=True)
data.replace({"시도": {"청라동": "인천광역시 서구 청라동"}}, inplace=True)
data.replace({"시도": {"두암동": "광주광역시 북구 두암동"}}, inplace=True)
data.replace({"시도": {"광주광역시북구": "광주광역시 북구"}}, inplace=True)
data.replace({"시도": {"지야동": "광주광역시 북구 지야동"}}, inplace=True)
data.replace({"시도": {"소촌동": "광주광역시 광산구 소촌동"}}, inplace=True)
data.replace({"시도": {"도천동": "광주광역시 광산구 도천동"}}, inplace=True)
data.replace({"시도": {"진곡동": "광주광역시 광산구 진곡동"}}, inplace=True)
data.replace({"시도": {"오선동": "광주광역시 광산구 오선동"}}, inplace=True)
data.replace({"시도": {"낭월동": "대전광역시 동구 낭월동"}}, inplace=True)
data.replace({"시도": {"장대동": "대전광역시 유성구 장대동"}}, inplace=True)
data.replace({"시도": {"대전광역시유성구": "대전광역시 유성구"}}, inplace=True)
data.replace({"시도": {"탑립동": "대전광역시 유성구 탑립동"}}, inplace=True)
data.replace({"시도": {"신동": "대전광역시 유성구 신동"}}, inplace=True)
data.replace({"시도": {"오정동": "대전광역시 대덕구 오정동"}}, inplace=True)

In [255]:
data.replace({"시도": {"용당동": "경상남도 양산시 용당동"}}, inplace=True)
data.replace({"시도": {"연산동": "광주광역시 광산구 연산동"}}, inplace=True)
data.replace({"시도": {"비산동": "대구광역시 서구 비산동"}}, inplace=True)
data.replace({"시도": {"이현동": "대구광역시 서구 이현동"}}, inplace=True)
data.replace({"시도": {"대화동": "대전광역시 대덕구 대화동"}}, inplace=True)
data.replace({"시도": {"신일동": "대전광역시 대덕구 신일동"}}, inplace=True)
data.replace({"시도": {"황성동": "울산광역시 남구 황성동"}}, inplace=True)
data.replace({"시도": {"달천동": "울산광역시 북구 달천동"}}, inplace=True)
data.replace({"시도": {"청량읍": "울산광역시 울주군 청량읍"}}, inplace=True)
data.replace({"시도": {"집현동": "세종특별자치시 집현동"}}, inplace=True)
data.replace({"시도": {"연동면": "세종특별자치시 연동면"}}, inplace=True)
data.replace({"시도": {"세종특별자치시부강면": "세종특별자치시 부강면"}}, inplace=True)
data.replace({"시도": {"시흥동": "경기도 성남시 수정구 시흥동"}}, inplace=True)
data.replace({"시도": {"상대원동": "경기도 성남시 중원구 상대원동"}}, inplace=True)
data.replace({"시도": {"안양동": "경기도 안양시 만안구 안양동"}}, inplace=True)
data.replace({"시도": {"춘의동": "경기도 부천시 원미구 춘의동"}}, inplace=True)
data.replace({"시도": {"도당동": "경기도 부천시 원미구 도당동"}}, inplace=True)
data.replace({"시도": {"역곡동": "경기도 부천시 원미구 역곡동"}}, inplace=True)
data.replace({"시도": {"삼정동": "경기도 부천시 오정구 삼정동"}}, inplace=True)
data.replace({"시도": {"내동": "경기도 부천시 오정구 내동"}}, inplace=True)
data.replace({"시도": {"세교동": "경기도 평택시 세교동"}}, inplace=True)
data.replace({"시도": {"청북읍": "경기도 평택시 청북읍"}}, inplace=True)
data.replace({"시도": {"진위면": "경기도 평택시 진위면"}}, inplace=True)
data.replace({"시도": {"지행동": "경기도 동두천시 지행동"}}, inplace=True)
data.replace({"시도": {"경기도안산시": "경기도 안산시"}}, inplace=True)
data.replace({"시도": {"팔곡이동": "경기도 안산시 상록구 팔곡이동"}}, inplace=True)
data.replace({"시도": {"성곡동": "경기도 안산시 단원구 성곡동"}}, inplace=True)

In [259]:
data.replace({"시도": {"원시동": "경기도 안산시 단원구 원시동"}}, inplace=True)
data.replace({"시도": {"목내동": "경기도 안산시 단원구 목내동"}}, inplace=True)
data.replace({"시도": {"갈매동": "경기도 구리시 갈매동"}}, inplace=True)
data.replace({"시도": {"인창동": "경기도 구리시 인창동"}}, inplace=True)
data.replace({"시도": {"오남읍": "경기도 남양주시 오남읍"}}, inplace=True)
data.replace({"시도": {"양산동": "경기도 오산시 양산동"}}, inplace=True)
data.replace({"시도": {"은행동": "경기도 시흥시 은행동"}}, inplace=True)
data.replace({"시도": {"매화동": "경기도 시흥시 매화동"}}, inplace=True)
data.replace({"시도": {"도창동": "경기도 시흥시 도창동"}}, inplace=True)
data.replace({"시도": {"정왕동": "경기도 시흥시 정왕동"}}, inplace=True)
data.replace({"시도": {"배곧동": "경기도 시흥시 배곧동"}}, inplace=True)
data.replace({"시도": {"당동": "경기도 군포시 당동"}}, inplace=True)
data.replace({"시도": {"경기도군포시": "경기도 군포시"}}, inplace=True)
data.replace({"시도": {"당정동": "경기도 군포시 당정동"}}, inplace=True)
data.replace({"시도": {"산본동": "경기도 군포시 산본동"}}, inplace=True)
data.replace({"시도": {"삼동": "경기도 의왕시 삼동"}}, inplace=True)
data.replace({"시도": {"경기도의왕시": "경기도 의왕시"}}, inplace=True)
data.replace({"시도": {"이동읍": "경기도 용인시 처인구 이동읍"}}, inplace=True)
data.replace({"시도": {"양지면": "경기도 용인시 처인구 양지면"}}, inplace=True)
data.replace({"시도": {"신갈동": "경기도 용인시 기흥구 신갈동"}}, inplace=True)
data.replace({"시도": {"영덕동": "경기도 용인시 기흥구 영덕동"}}, inplace=True)
data.replace({"시도": {"풍덕천동": "경기도 용인시 수지구 풍덕천동"}}, inplace=True)
data.replace({"시도": {"검산동": "경기도 파주시 검산동"}}, inplace=True)
data.replace({"시도": {"신촌동": "경기도 파주시 신촌동"}}, inplace=True)
data.replace({"시도": {"법원읍": "경기도 파주시 법원읍"}}, inplace=True)
data.replace({"시도": {"월롱면": "경기도 파주시 월롱면"}}, inplace=True)

In [262]:
data.replace({"시도": {"탄현면": "경기도 파주시 탄현면"}}, inplace=True)
data.replace({"시도": {"운양동": "경기도 김포시 운양동"}}, inplace=True)
data.replace({"시도": {"구래동": "경기도 김포시 구래동"}}, inplace=True)
data.replace({"시도": {"통진읍": "경기도 김포시 통진읍"}}, inplace=True)
data.replace({"시도": {"양촌읍": "경기도 김포시 양촌읍"}}, inplace=True)
data.replace({"시도": {"대곶면": "경기도 김포시 대곶면"}}, inplace=True)
data.replace({"시도": {"월곶면": "경기도 김포시 월곶면"}}, inplace=True)
data.replace({"시도": {"영천동": "경기도 화성시 영천동"}}, inplace=True)
data.replace({"시도": {"새솔동": "경기도 화성시 새솔동"}}, inplace=True)
data.replace({"시도": {"우정읍": "경기도 화성시 우정읍"}}, inplace=True)
data.replace({"시도": {"매송면": "경기도 화성시 매송면"}}, inplace=True)
data.replace({"시도": {"마도면": "경기도 화성시 마도면"}}, inplace=True)
data.replace({"시도": {"서신면": "경기도 화성시 서신면"}}, inplace=True)
data.replace({"시도": {"정남면": "경기도 화성시 정남면"}}, inplace=True)
data.replace({"시도": {"초월읍": "경기도 광주시 초월읍"}}, inplace=True)
data.replace({"시도": {"남면": "경기도 양주시 남면"}}, inplace=True)
data.replace({"시도": {"자작동": "경기도 포천시 자작동"}}, inplace=True)
data.replace({"시도": {"가산면": "경기도 포천시 가산면"}}, inplace=True)
data.replace({"시도": {"화현면": "경기도 포천시 화현면"}}, inplace=True)
data.replace({"시도": {"현도면": "충청북도 청주시 서원구 현도면"}}, inplace=True)
data.replace({"시도": {"내수읍": "충청북도 청주시 청원구 내수읍"}}, inplace=True)
data.replace({"시도": {"오창읍": "충청북도 청주시 청원구 오창읍"}}, inplace=True)
data.replace({"시도": {"금성면": "충청북도 제천시 금성면"}}, inplace=True)
data.replace({"시도": {"충청북도영동군": "충청북도 영동군"}}, inplace=True)
data.replace({"시도": {"이월면": "충청북도 진천군 이월면"}}, inplace=True)
data.replace({"시도": {"안서동": "충청남도 천안시 동남구 안서동"}}, inplace=True)
data.replace({"시도": {"충청남도천안시": "충청남도 천안시"}}, inplace=True)

In [265]:
data.replace({"시도": {"성거읍": "충청남도 천안시 서북구 성거읍"}}, inplace=True)
data.replace({"시도": {"둔포면": "충청남도 아산시 둔포면"}}, inplace=True)
data.replace({"시도": {"성동면": "충청남도 논산시 성동면"}}, inplace=True)
data.replace({"시도": {"벌곡면": "충청남도 논산시 벌곡면"}}, inplace=True)
data.replace({"시도": {"면천면": "충청남도 당진시 면천면"}}, inplace=True)
data.replace({"시도": {"장항읍": "충청남도 서천군 장항읍"}}, inplace=True)
data.replace({"시도": {"종천면": "충청남도 서천군 종천면"}}, inplace=True)
data.replace({"시도": {"오가면": "충청남도 예산군 오가면"}}, inplace=True)
data.replace({"시도": {"평여동": "전라남도 여수시 평여동"}}, inplace=True)
data.replace({"시도": {"전라남도순천시": "전라남도 순천시"}}, inplace=True)
data.replace({"시도": {"대전면": "전라남도 담양군 대전면"}}, inplace=True)
data.replace({"시도": {"전라남도고흥군": "전라남도 고흥군"}}, inplace=True)
data.replace({"시도": {"장흥읍": "전라남도 장흥군 장흥읍"}}, inplace=True)
data.replace({"시도": {"화원면": "전라남도 해남군 화원면"}}, inplace=True)
data.replace({"시도": {"삼호읍": "전라남도 영암군 삼호읍"}}, inplace=True)
data.replace({"시도": {"전라남도영암군": "전라남도 영암군"}}, inplace=True)
data.replace({"시도": {"전라남도장성군": "전라남도 장성군"}}, inplace=True)
data.replace({"시도": {"경상북도김천시": "경상북도 김천시"}}, inplace=True)
data.replace({"시도": {"개령면": "경상북도 김천시 개령면"}}, inplace=True)
data.replace({"시도": {"공단동": "경상북도 구미시 공단동"}}, inplace=True)
data.replace({"시도": {"산동읍": "경상북도 구미시 산동읍"}}, inplace=True)
data.replace({"시도": {"상줄동": "경상북도 영주시 상줄동"}}, inplace=True)
data.replace({"시도": {"금호읍": "경상북도 영천시 금호읍"}}, inplace=True)
data.replace({"시도": {"성산면": "경상북도 고령군 성산면"}}, inplace=True)
data.replace({"시도": {"경상북도성주군": "경상북도 성주군"}}, inplace=True)
data.replace({"시도": {"선남면": "경상북도 성주군 선남면"}}, inplace=True)
data.replace({"시도": {"호명읍": "경상북도 예천군 호명읍"}}, inplace=True)

In [268]:
data.replace({"시도": {"평해읍": "경상북도 울진군 평해읍"}}, inplace=True)
data.replace({"시도": {"죽변면": "경상북도 울진군 죽변면"}}, inplace=True)
data.replace({"시도": {"봉암동": "경상남도 창원시 마산회원구 봉암동"}}, inplace=True)
data.replace({"시도": {"내서읍": "경상남도 창원시 마산회원구 내서읍"}}, inplace=True)
data.replace({"시도": {"진성면": "경상남도 진주시 진성면"}}, inplace=True)
data.replace({"시도": {"도산면": "경상남도 통영시 도산면"}}, inplace=True)
data.replace({"시도": {"풍유동": "경상남도 김해시 풍유동"}}, inplace=True)
data.replace({"시도": {"주촌면": "경상남도 김해시 주촌면"}}, inplace=True)
data.replace({"시도": {"진례면": "경상남도 김해시 진례면"}}, inplace=True)
data.replace({"시도": {"한림면": "경상남도 김해시 한림면"}}, inplace=True)
data.replace({"시도": {"대동면": "경상남도 김해시 대동면"}}, inplace=True)
data.replace({"시도": {"사등면": "경상남도 거제시 사등면"}}, inplace=True)
data.replace({"시도": {"북정동": "경상남도 양산시 북정동"}}, inplace=True)
data.replace({"시도": {"경상남도양산시": "경상남도 양산시"}}, inplace=True)
data.replace({"시도": {"호계동": "경상남도 양산시 호계동"}}, inplace=True)
data.replace({"시도": {"명동": "경상남도 양산시 명동"}}, inplace=True)
data.replace({"시도": {"가야읍": "경상남도 함안군 가야읍"}}, inplace=True)
data.replace({"시도": {"칠서면": "경상남도 함안군 칠서면"}}, inplace=True)
data.replace({"시도": {"대지면": "경상남도 창녕군 대지면"}}, inplace=True)
data.replace({"시도": {"생비량면": "경상남도 산청군 생비량면"}}, inplace=True)
data.replace({"시도": {"제주도": "제주특별자치도"}}, inplace=True)
data.replace({"시도": {"제주특별자치": "제주특별자치도"}}, inplace=True)
data.replace({"시도": {"제주특별자치도서귀포시": "제주특별자치도 서귀포시"}}, inplace=True)
data.replace({"시도": {"단구동": "강원특별자치도 원주시 단구동"}}, inplace=True)
data.replace({"시도": {"대전동": "강원특별자치도 강릉시 대전동"}}, inplace=True)
data.replace({"시도": {"강원특별자치도강릉시": "강원특별자치도 강릉시"}}, inplace=True)
data.replace({"시도": {"강원특별자치도동해시": "강원특별자치도 동해시"}}, inplace=True)

In [271]:
data.replace({"시도": {"강원특별자치도태백시": "강원특별자치도 태백시"}}, inplace=True)
data.replace({"시도": {"오식도동": "전북특별자치도 군산시 오식도동"}}, inplace=True)
data.replace({"시도": {"옥산면": "전북특별자치도 군산시 오식도동"}}, inplace=True)
data.replace({"시도": {"하북동": "전북특별자치도 정읍시 하북동"}}, inplace=True)
data.replace({"시도": {"고부면": "전북특별자치도 정읍시 고부면"}}, inplace=True)
data.replace({"시도": {"노암동": "전북특별자치도 남원시 노암동"}}, inplace=True)
data.replace({"시도": {"봉동읍": "전북특별자치도 완주군 봉동읍"}}, inplace=True)
data.replace({"시도": {"고수면": "전북특별자치도 고창군 고수면"}}, inplace=True)


In [272]:
data["전체주소"]=data["시도"]+" "+data["시군구"]+" "+data["읍면동"].fillna("")+" "+data["상세주소"].fillna("")
data[["시도", "시군구", "읍면동", "상세주소"]] = data["전체주소"].apply(split_full_address)

In [273]:
data.시도.unique()

array(['서울특별시', '인천광역시', '부산광역시', '경상남도', '광주광역시', '대구광역시', '대전광역시',
       '울산광역시', '세종특별자치시', '충청남도', '경기도', '풍산동', '충청북도', '전라남도', '경상북도',
       '제주특별자치도', '강원특별자치도', '전북특별자치도', '용정동'], dtype=object)

전처리 하지 못한 주소는 제거

In [280]:
data=data.loc[~data["시도"].isin(["풍산동", "용정동"])]

In [281]:
data.시도.unique()

array(['서울특별시', '인천광역시', '부산광역시', '경상남도', '광주광역시', '대구광역시', '대전광역시',
       '울산광역시', '세종특별자치시', '충청남도', '경기도', '충청북도', '전라남도', '경상북도',
       '제주특별자치도', '강원특별자치도', '전북특별자치도'], dtype=object)

In [285]:
data.isna().sum()

시도        0
시군구       0
읍면동       0
상세주소    571
회사명       0
공장규모      0
업종명       0
전체주소      0
dtype: int64

## 네이버 지도 API 좌표

In [283]:
data=data[["시도","시군구","읍면동","상세주소","회사명","공장규모","업종명","전체주소"]]

In [284]:
# 기존 DataFrame의 '전체주소' 컬럼을 리스트로 변환
addresses = data['전체주소'].tolist()

# ThreadPoolExecutor를 이용해 동시 실행
with ThreadPoolExecutor(max_workers=100) as executor:
    results = list(executor.map(get_coordinates_from_address, addresses))

# 결과를 DataFrame으로 변환 후 기존 DataFrame과 결합
df_coordinates = pd.DataFrame(results)
df_final = pd.concat([data.reset_index(drop=True), df_coordinates], axis=1)
print(df_final)

             시도  시군구  읍면동          상세주소          회사명 공장규모  \
0         서울특별시  종로구  창성동        87-1번지     남일문화 (주)  소기업   
1         서울특별시  종로구  통의동       35-69번지    (주)에취.알.디  소기업   
2         서울특별시  종로구  통의동        35-6번지  (주)코리아쉬핑가제트  소기업   
3         서울특별시  종로구  적선동       107-1번지         삼영DP  소기업   
4         서울특별시  종로구  적선동        20-0번지        중앙인쇄사  소기업   
...         ...  ...  ...           ...          ...  ...   
217103  전북특별자치도  부안군  줄포면  장동리 741-15번지   제이페이퍼 주식회사  소기업   
217104  전북특별자치도  부안군  줄포면  신리 352-59 번지  유한회사 줄포하늘수산  소기업   
217105  전북특별자치도  부안군  줄포면   파산리 756-6번지    남부안농업협동조합  소기업   
217106  전북특별자치도  부안군  줄포면   대동리 10-14번지        지산정미소  소기업   
217107  전북특별자치도  부안군  위도면    식도리 1-24번지         금영수산  소기업   

                             업종명                          전체주소    Latitude  \
0                    경 인쇄업 외 2 종          서울특별시 종로구 창성동 87-1번지  37.5814832   
1                     전시용 모형 제조업         서울특별시 종로구 통의동 35-69번지  37.5780443   
2                일반 서적 출판업 외 3 종 

In [286]:
df_final.isna().sum()

시도              0
시군구             0
읍면동             0
상세주소          571
회사명             0
공장규모            0
업종명             0
전체주소            0
Latitude     1744
Longitude    1744
dtype: int64

결측치가 생긴 좌표는 제거하자

In [287]:
df_final.dropna(subset=["Latitude"], inplace=True)

In [288]:
df_final.isna().sum()

시도             0
시군구            0
읍면동            0
상세주소         569
회사명            0
공장규모           0
업종명            0
전체주소           0
Latitude       0
Longitude      0
dtype: int64

## 전처리 마무리

In [289]:
df_final[["Latitude","Longitude"]]=df_final[["Latitude","Longitude"]].apply(pd.to_numeric, errors='coerce')

In [290]:
df_final.drop(columns=["전체주소"], axis=1, inplace=True)

In [291]:
df_final.columns=["County","City","Town","Address","Factory_Name","Factory_Size","Industry_Type","Latitude","Longitude"]

In [ ]:
df_final.to_csv("결과데이터/전국공장정보_2025.csv", index=False, encoding="utf-8-sig")

# 18. 전국 농산물 직거래 판매장 정보

In [ ]:
data=pd.read_csv("원본데이터/한국농수산식품유통공사_농산물 직거래 판매장 정보_20240812.csv", encoding="cp949")
data=data[["판매장 이름", "주소"]]
data.head(1)

,판매장 이름,주소
0,대전로컬푸드라온아띠협동조합,대전 서구 남선로 66


## 주소 분할

In [45]:
data[["County", "City", "Address"]] = data["주소"].apply(split_full_address_3)
data.County.unique()

array(['대전', '전북', '경남', '광주', '경기도남양주시', '경기도', '충남', '전남', '경북', '강원도',
       '청주시', '제주특별자치도', '강원', '충북', '경기', '전라남도', '충청북도', '대구', '진주시',
       '경상북도', '안성시', '서울', '부산', '울산', '세종특별자치시', '대구광역시', '전라북도',
       '울산광역시', '목포시', '인천광역시'], dtype=object)

In [46]:
data.isna().sum()

판매장 이름     0
주소         0
County     0
City       0
Address    0
dtype: int64

주소에 오류가 있는 것을 고치자

In [48]:
data.replace({"County": {"대전": "대전광역시"}}, inplace=True)
data.replace({"County": {"전북": "전라북도"}}, inplace=True)
data.replace({"County": {"경남": "경상남도"}}, inplace=True)
data.replace({"County": {"광주": "광주광역시"}}, inplace=True)
data.replace({"County": {"경기도남양주시": "경기도 남양주시"}}, inplace=True)
data.replace({"County": {"충남": "충청남도"}}, inplace=True)
data.replace({"County": {"전남": "전라남도"}}, inplace=True)
data.replace({"County": {"경북": "경상북도"}}, inplace=True)
data.replace({"County": {"청주시": "충청북도 청주시"}}, inplace=True)
data.replace({"County": {"강원": "강원도"}}, inplace=True)
data.replace({"County": {"충북": "충청북도"}}, inplace=True)
data.replace({"County": {"경기": "경기도"}}, inplace=True)
data.replace({"County": {"대구": "대구광역시"}}, inplace=True)
data.replace({"County": {"진주시": "경상남도 진주시"}}, inplace=True)
data.replace({"County": {"안성시": "경기도 안성시"}}, inplace=True)
data.replace({"County": {"서울": "서울특별시"}}, inplace=True)
data.replace({"County": {"부산": "부산광역시"}}, inplace=True)
data.replace({"County": {"울산": "울산광역시"}}, inplace=True)
data.replace({"County": {"목포시": "전라남도 목포시"}}, inplace=True)

다시 합치고 재분할

In [49]:
data["주소"]=data["County"]+" "+data["City"]+" "+data["Address"].fillna("")
data[["County", "City", "Address"]] = data["주소"].apply(split_full_address_3)

In [50]:
data.County.unique()

array(['대전광역시', '전라북도', '경상남도', '광주광역시', '경기도', '충청남도', '전라남도', '경상북도',
       '강원도', '충청북도', '제주특별자치도', '대구광역시', '서울특별시', '부산광역시', '울산광역시',
       '세종특별자치시', '인천광역시'], dtype=object)

In [51]:
data.isna().sum()

판매장 이름     0
주소         0
County     0
City       0
Address    0
dtype: int64

## 네이버 지도 API 좌표

In [52]:
#주소 칼럼 입력
addresses = data['주소'].tolist()

# ThreadPoolExecutor를 이용해 동시 실행
##하나씩 실행하면 느리다
with ThreadPoolExecutor(max_workers=100) as executor: #한번에 100개
    results = list(executor.map(get_coordinates_from_address, addresses))

# 결과를 DataFrame으로 변환 후 기존 DataFrame과 결합
df_coordinates = pd.DataFrame(results)
df_final = pd.concat([data.reset_index(drop=True), df_coordinates], axis=1)
print(df_final)

                    판매장 이름                         주소   County  City  \
0           대전로컬푸드라온아띠협동조합            대전광역시 서구 남선로 66    대전광역시    서구   
1             고모네장터로컬푸드직매장       전라북도 정읍시 북면 정읍북로 603     전라북도   정읍시   
2    진주우리먹거리협동조합 진주텃밭(금산점)      경상남도 진주시 금산면 금산순환로 61     경상남도   진주시   
3         광주농협 로컬푸드직매장 매곡점            광주광역시 북구 매곡로 50    광주광역시    북구   
4            진접농협 로컬푸드 직매장      경기도 남양주시 진접읍 해밀예당1로 8      경기도  남양주시   
..                     ...                        ...      ...   ...   
237                올바른농민상회  제주특별자치도 제주시 애월읍 일주서로 6585  제주특별자치도   제주시   
238       양남농협하나로마트로컬푸드판매장     경상북도 경주시 양남면 해변공원길 103     경상북도   경주시   
239    대전 충청 한밭가득 로컬푸드 직매장           대전광역시 서구 계룡로 598    대전광역시    서구   
240           반월농협 로컬푸드직매장     경기도 안산시 상록구 해안로 1190-1      경기도   안산시   
241         봉산농협 쌍교로컬푸드직매장      전라남도 담양군 봉산면 송강정로 208     전라남도   담양군   

            Address    Latitude    Longitude  
0            남선로 66  36.3452575  127.3966837  
1       북면 정읍북로 603  35.6118256  126.8920

In [53]:
df_final.isna().sum()

판매장 이름       0
주소           0
County       0
City         0
Address      0
Latitude     3
Longitude    3
dtype: int64

좌표에 오류난 건 수작업으로 기입하자

In [54]:
df_final[["Latitude","Longitude"]]=df_final[["Latitude","Longitude"]].apply(pd.to_numeric, errors='coerce')
df_final.dtypes

판매장 이름        object
주소            object
County        object
City          object
Address       object
Latitude     float64
Longitude    float64
dtype: object

In [58]:
df_final[df_final["Latitude"].isna()|df_final["Longitude"].isna()]

,판매장 이름,주소,County,City,Address,Latitude,Longitude
5,협동조합 함박꽃웃음,경기도 의왕시 651-13,경기도,의왕시,651-13,NaN,NaN
92,화성로컬푸드직매장 금곡점,경기도 화성시 동탄면 금곡로 203,경기도,화성시,동탄면 금곡로 203,NaN,NaN
145,경주농특산물판매장 신경주역점,경상북도 경주시 건천읍 신경주역로 80,경상북도,경주시,건천읍 신경주역로 80,NaN,NaN


In [59]:
df_final.loc[5, ["Latitude","Longitude"]]=[37.387711, 126.993689]
df_final.loc[92, ["Latitude","Longitude"]]=[37.178433, 127.079047]
df_final.loc[145, ["Latitude","Longitude"]]=[35.799627, 129.140294]


In [60]:
df_final.isna().sum()

판매장 이름       0
주소           0
County       0
City         0
Address      0
Latitude     0
Longitude    0
dtype: int64

## 마무리 전처리 후 저장

In [61]:
df_final.rename(columns={"판매장 이름":"Local_Store_Name"}, inplace=True)
df_final.drop(columns=["주소"], axis=1, inplace=True)

In [ ]:
df_final.to_csv("결과데이터/전국농산물직거래판매장정보.csv", index=False, encoding="cp949")

# 24. 전국 **공시지가, 실거래가** 정보

주소의 최소단위 - 같은 읍면동(리)에서의 공시지가, 실거래가의 평균을 구하자

## 1) 공시지가

In [ ]:
data1=pd.read_csv("원본데이터/appraised_value_of_land.csv",encoding='EUC-KR')
data1.head(1)

,법정동명,지번,공시지가
0,서울특별시 종로구 청운동,1,5150000


In [3]:
data1.isna().sum()

법정동명    0
지번      1
공시지가    0
dtype: int64

In [4]:
data1.dtypes

법정동명    object
지번      object
공시지가     int64
dtype: object

In [5]:
data1[["법정동명","지번"]].value_counts()

법정동명                 지번   
전라남도 구례군 용방면 신지리     3        4
전북특별자치도 진안군 부귀면 수항리  157      4
충청남도 부여군 은산면 나령리     14-1     4
경기도 하남시 감이동          5-1      4
경기도 남양주시 조안면 능내리     12-22    4
                             ..
경상북도 경산시 내동          211      1
                     210      1
                     21       1
                     209-2    1
충청북도 충주시 호암동         999      1
Name: count, Length: 34456936, dtype: int64

같은 주소, 번지수가 있는게 있다 (호수는 다를 수 있다)  
획일적인 분석을 위해 하나로 통합하자.  
이때 공시지가는 평균을 낸다

In [6]:
data1=data1.groupby(["법정동명", "지번"], as_index=False)["공시지가"].mean()

## 2) 실거래가

In [ ]:
data2=pd.read_csv("원본데이터/actual_transaction_price.csv",encoding='EUC-KR')
data2.head(1)

,시군구,번지,거래금액(만원),주택 유형
0,부산광역시 부산진구 양정동,458-20,"35,500",아파트


In [41]:
data2.isna().sum()

시군구         0
번지          2
거래금액(만원)    0
주택 유형       0
dtype: int64

In [42]:
data2.dtypes

시군구         object
번지          object
거래금액(만원)    object
주택 유형       object
dtype: object

거래금액(만원)이 문자열이므로 이를 숫자형으로 바꿔준다.  
이때 쉼표로 인해 전처리 과정이 필요하다

In [43]:
data2["거래금액(만원)"] = data2["거래금액(만원)"].str.replace(",", "", regex=False).astype(float)
data2.dtypes

시군구          object
번지           object
거래금액(만원)    float64
주택 유형        object
dtype: object

## 3) 공시지가, 실거래가 합치기

### i. 주소 분할 (주소의 오류를 줄이기 위해)

공시지가

기존의 주소 분할 함수를 사용하면 너무 오래 걸린다.  
따라서 고정적으로 4개로 분할하는 코드를 사용한다. (상세주소가 없는 경우이므로 가능하다)

In [11]:
split_cols = data1["법정동명"].astype(str).str.strip().str.split(n=3, expand=True) #빈칸 세개로 분할

#데이터가 두개, 세개로 분할될경우 남는것에 결측치를 넣어줌
for i in range(4 - split_cols.shape[1]):
    split_cols[split_cols.shape[1] + i] = pd.NA
    
split_cols.columns = ["County", "City", "Town", "Village"]
data1[["County", "City", "Town", "Village"]] = split_cols

In [12]:
data1.isna().sum()

법정동명             0
지번               0
공시지가             0
County           0
City             0
Town          5331
Village    6105740
dtype: int64

In [15]:
data1.County.unique()

array(['세종특별자치시', '강원특별자치도', '경기도', '경상남도', '경상북도', '광주광역시', '대구광역시',
       '대전광역시', '부산광역시', '서울특별시', '울산광역시', '인천광역시', '전라남도', '전북특별자치도',
       '제주특별자치도', '충청남도', '충청북도'], dtype=object)

문제가 없어 보인다

실거래가

In [44]:
data2[["County", "City", "Town", "Village"]] = data2["시군구"].apply(split_full_address)

In [45]:
data2.isna().sum()

시군구              0
번지               2
거래금액(만원)         0
주택 유형            0
County           0
City             0
Town          3891
Village     380427
dtype: int64

In [46]:
data2.County.unique()

array(['부산광역시', '인천광역시', '대구광역시', '서울특별시', '광주광역시', '경기도', '세종특별자치시',
       '대전광역시', '울산광역시', '전북특별자치도', '경상남도', '강원특별자치도', '제주특별자치도', '전라남도',
       '충청북도', '경상북도', '충청남도'], dtype=object)

In [47]:
data2[data2.Town.isna()].head()

,시군구,번지,거래금액(만원),주택 유형,County,City,Town,Village
314,세종특별자치시 종촌동,697,65000.0,아파트,세종특별자치시,종촌동,<NA>,<NA>
315,세종특별자치시 종촌동,654,37500.0,아파트,세종특별자치시,종촌동,<NA>,<NA>
316,세종특별자치시 종촌동,654,37500.0,아파트,세종특별자치시,종촌동,<NA>,<NA>
319,세종특별자치시 종촌동,690,53000.0,아파트,세종특별자치시,종촌동,<NA>,<NA>
389,세종특별자치시 아름동,1282,54600.0,아파트,세종특별자치시,아름동,<NA>,<NA>


오류가 없는 것 같다

### ii. 같은 주소, 번지수끼리 공시지가, 실거래가 결합하기

In [48]:
data2["주택 유형"].unique()

array(['아파트', '연립/다세대', '단독/다가구'], dtype=object)

주택 유형이 '단독/다가구'인것의 번지수가 별표(*)로 검열되어있다.  
따라서 이것들을 제외한것만 공시지가 데이터랑 결합한다

In [66]:
data2_iso=data2[data2["주택 유형"]=="단독/다가구"].copy()
data2_apt=data2[data2["주택 유형"].isin(["아파트","연립/다세대"])].copy()

In [67]:
print(data2.shape)
print(data2_iso.shape)
print(data2_apt.shape)

(597820, 8)
(45062, 8)
(552758, 8)


In [68]:
data2_iso.head()

,시군구,번지,거래금액(만원),주택 유형,County,City,Town,Village
552758,전북특별자치도 고창군 신림면 법지리,3**,16300.0,단독/다가구,전북특별자치도,고창군,신림면,법지리
552759,경상남도 합천군 초계면 대동리,2**,2500.0,단독/다가구,경상남도,합천군,초계면,대동리
552760,경상남도 창원시 마산회원구 회성동,4**,8800.0,단독/다가구,경상남도,창원시,마산회원구,회성동
552761,경상북도 영덕군 축산면 경정리,3**,1650.0,단독/다가구,경상북도,영덕군,축산면,경정리
552762,경상북도 상주시 모동면 신흥리,1**,5800.0,단독/다가구,경상북도,상주시,모동면,신흥리


In [70]:
data2_iso.isna().sum()

시군구             0
번지              0
거래금액(만원)        0
주택 유형           0
County          0
City            0
Town            5
Village     16763
dtype: int64

In [69]:
data2_apt.head()

,시군구,번지,거래금액(만원),주택 유형,County,City,Town,Village
0,부산광역시 부산진구 양정동,458-20,35500.0,아파트,부산광역시,부산진구,양정동,<NA>
1,인천광역시 미추홀구 용현동,627-79,26900.0,아파트,인천광역시,미추홀구,용현동,<NA>
2,대구광역시 북구 태전동,1186,33000.0,아파트,대구광역시,북구,태전동,<NA>
3,서울특별시 서대문구 남가좌동,385,118000.0,아파트,서울특별시,서대문구,남가좌동,<NA>
4,인천광역시 연수구 연수동,536,39000.0,아파트,인천광역시,연수구,연수동,<NA>


In [71]:
data2_apt.isna().sum()

시군구              0
번지               2
거래금액(만원)         0
주택 유형            0
County           0
City             0
Town          3886
Village     363664
dtype: int64

실거래가도 같은 주소, 번지끼리 중복된게 있으므로 평균내주자

In [57]:
data2_apt[["County", "City", "Town", "Village", "번지", "주택 유형"]].value_counts()

County  City  Town   Village  번지      주택 유형 
충청남도    아산시   배방읍    공수리      1730    아파트       398
        천안시   서북구    두정동      2078    아파트       333
              동남구    신방동      920     아파트       297
경기도     안양시   만안구    안양동      1393    아파트       286
경상남도    창원시   마산합포구  월영동      738     아파트       281
                                               ... 
경기도     화성시   봉담읍    동화리      431-4   아파트         1
                              441-8   연립/다세대      1
                              473-4   연립/다세대      1
        부천시   오정구    고강동      359-15  아파트         1
경상남도    진주시   명석면    우수리      102-1   아파트         1
Name: count, Length: 26369, dtype: int64

In [56]:
data2_iso[["County", "City", "Town", "Village", "번지", "주택 유형"]].value_counts()

County   City  Town  Village  번지    주택 유형 
경상북도     포항시   북구    장성동      1***  단독/다가구    44
경기도      안산시   단원구   선부동      1***  단독/다가구    43
전북특별자치도  전주시   완산구   효자동3가    1***  단독/다가구    40
경상북도     포항시   남구    송도동      4**   단독/다가구    37
충청북도     청주시   흥덕구   가경동      6**   단독/다가구    36
                                              ..
경상북도     봉화군   봉화읍   유곡리      5*    단독/다가구     1
                     해저리      7*    단독/다가구     1
                     화천리      1**   단독/다가구     1
               상운면   가곡리      1***  단독/다가구     1
충청북도     충주시   중앙탑면  하구암리     7*    단독/다가구     1
Name: count, Length: 17793, dtype: int64

그룹핑하기 전, 결측치를 다른 값으로 채우기 (결측치가 있으면 이것을 무시하기 때문에)

In [ ]:
data2_iso[["Town", "Village"]] = data2_iso[["Town", "Village"]].fillna("None")
data2_apt[["Town", "Village"]] = data2_apt[["Town", "Village"]].fillna("None")


In [73]:
data2_iso.isna().sum()

시군구         0
번지          0
거래금액(만원)    0
주택 유형       0
County      0
City        0
Town        0
Village     0
dtype: int64

In [74]:
data2_iso = data2_iso.groupby(["County", "City", "Town", "Village", "번지", "주택 유형"], as_index=False)["거래금액(만원)"].mean()
data2_apt = data2_apt.groupby(["County", "City", "Town", "Village", "번지", "주택 유형"], as_index=False)["거래금액(만원)"].mean()

In [75]:
data2_apt.head()

,County,City,Town,Village,번지,주택 유형,거래금액(만원)
0,강원특별자치도,강릉시,견소동,None,202,아파트,19770.454545
1,강원특별자치도,강릉시,견소동,None,289,아파트,25819.354839
2,강원특별자치도,강릉시,교동,None,1034,아파트,15260.000000
3,강원특별자치도,강릉시,교동,None,1035-1,아파트,30500.000000
4,강원특별자치도,강릉시,교동,None,1429-1,아파트,8800.000000


In [78]:
data2_iso.head()

,County,City,Town,Village,번지,주택 유형,거래금액(만원)
0,강원특별자치도,강릉시,강동면,모전리,1**,단독/다가구,7440.0
1,강원특별자치도,강릉시,강동면,모전리,2**,단독/다가구,7100.0
2,강원특별자치도,강릉시,강동면,모전리,3**,단독/다가구,11000.0
3,강원특별자치도,강릉시,강동면,상시동리,1**,단독/다가구,33778.0
4,강원특별자치도,강릉시,강동면,상시동리,2**,단독/다가구,15000.0


먼저 단독주택을 제외한 실거래가랑 공시지가랑 결합하자

In [ ]:
#데이터 결합의 획일성을 위해
data1.rename(columns={"지번":"번지"}, inplace=True)
data1.columns

Index(['법정동명', '번지', '공시지가', 'County', 'City', 'Town', 'Village'], dtype='object')

In [80]:
data1.isna().sum()

법정동명             0
번지               0
공시지가             0
County           0
City             0
Town          5331
Village    6105740
dtype: int64

결측치가 있으면 결합이 안되기 때문에 다른 값으로 변환 후 진행

In [81]:
data1[["Town", "Village"]] = data1[["Town", "Village"]].fillna("None")

결합

In [87]:
temp=pd.merge(data1, data2_apt, how="inner", on=["County", "City", "Town", "Village", "번지"])

In [88]:
print(data1.shape)
print(data2_apt.shape)
print(temp.shape)

(34456936, 7)
(83366, 7)
(83014, 9)


잘 되었다

In [89]:
temp.head()

,법정동명,번지,공시지가,County,City,Town,Village,주택 유형,거래금액(만원)
0,세종특별자치시 반곡동,1035,1460000.0,세종특별자치시,반곡동,None,None,아파트,38954.545455
1,세종특별자치시 반곡동,841,4198000.0,세종특별자치시,반곡동,None,None,연립/다세대,13000.000000
2,세종특별자치시 반곡동,851,1656000.0,세종특별자치시,반곡동,None,None,아파트,72790.476190
3,세종특별자치시 반곡동,852,1672000.0,세종특별자치시,반곡동,None,None,아파트,67078.787879
4,세종특별자치시 반곡동,857,1821000.0,세종특별자치시,반곡동,None,None,아파트,58656.875000


### iii. 읍면동(리)단위, 주택유형별로 공시지가, 실거래가 평균내기

위의 단독주택의 경우 따로 평균 낸 다음에 concat하기

In [90]:
temp.isna().sum()

법정동명        0
번지          0
공시지가        0
County      0
City        0
Town        0
Village     0
주택 유형       0
거래금액(만원)    0
dtype: int64

In [ ]:
temp=temp.groupby(["County", "City", "Town", "Village", "주택 유형"], as_index=False)[["공시지가", "거래금액(만원)"]].mean()
temp.head()

,County,City,Town,Village,주택 유형,공시지가,거래금액(만원)
0,강원특별자치도,강릉시,견소동,None,아파트,629050.000000,22794.904692
1,강원특별자치도,강릉시,교동,None,아파트,498478.333333,20658.683003
2,강원특별자치도,강릉시,교동,None,연립/다세대,365216.666667,14333.333333
3,강원특별자치도,강릉시,남문동,None,아파트,546700.000000,17100.000000
4,강원특별자치도,강릉시,내곡동,None,아파트,475427.272727,14772.485871
...,...,...,...,...,...,...,...
6781,충청북도,충주시,칠금동,None,아파트,473450.000000,18221.948288
6782,충청북도,충주시,칠금동,None,연립/다세대,411900.000000,6500.000000
6783,충청북도,충주시,풍동,None,아파트,148100.000000,3596.607143
6784,충청북도,충주시,호암동,None,아파트,469061.538462,21754.533597


In [96]:
data2_iso=data2_iso.groupby(["County", "City", "Town", "Village", "주택 유형"], as_index=False)["거래금액(만원)"].mean()
data2_iso.head()

,County,City,Town,Village,주택 유형,거래금액(만원)
0,강원특별자치도,강릉시,강동면,모전리,단독/다가구,8513.333333
1,강원특별자치도,강릉시,강동면,상시동리,단독/다가구,18194.500000
2,강원특별자치도,강릉시,강동면,심곡리,단독/다가구,11975.250000
3,강원특별자치도,강릉시,강동면,안인리,단독/다가구,15100.000000
4,강원특별자치도,강릉시,강동면,안인진리,단독/다가구,55000.000000


두 데이터를 concat

In [100]:
final_df=pd.concat([temp, data2_iso], axis=0)

In [102]:
final_df.isna().sum()

County          0
City            0
Town            0
Village         0
주택 유형           0
공시지가        11558
거래금액(만원)        0
dtype: int64

위에서 Town, Village의 결측치를 "None"으로 바꾼 것을 다시 결측치로 바꾸기

In [104]:
final_df.replace({"Town": {"None": pd.NA}}, inplace=True)
final_df.replace({"Village": {"None": pd.NA}}, inplace=True)
final_df.isna().sum()

County          0
City            0
Town           20
Village      5246
주택 유형           0
공시지가        11558
거래금액(만원)        0
dtype: int64

### iv. 마무리 전처리

금액의 소숫점 아래 자리가 너무 길다

In [109]:
final_df["공시지가"]=final_df["공시지가"].round()
final_df["거래금액(만원)"]=final_df["거래금액(만원)"].round()

칼럼명 영어로 바꾸기

In [112]:
final_df.rename(columns={"주택 유형":"Housing_Type", "공시지가":"Official_Land_Value", "거래금액(만원)":"Actual_Land_Value"}, inplace=True)

In [ ]:
final_df.head()

,County,City,Town,Village,Housing_Type,Official_Land_Value,Actual_Land_Value
0,강원특별자치도,강릉시,견소동,<NA>,아파트,629050.0,22795.0
1,강원특별자치도,강릉시,교동,<NA>,아파트,498478.0,20659.0
2,강원특별자치도,강릉시,교동,<NA>,연립/다세대,365217.0,14333.0
3,강원특별자치도,강릉시,남문동,<NA>,아파트,546700.0,17100.0
4,강원특별자치도,강릉시,내곡동,<NA>,아파트,475427.0,14772.0


저장하기

In [ ]:
final_df.to_csv("결과데이터/전국주택가격정보.csv", index=False, encoding="cp949")

# 25. 전국 농지매물 정보

동적 크롤링 활용

부산광역시, 대구광역시, 인천광역시, 광주광역시, 세종특별자치시, 제주특별자치도   
울산광역시, 경기도, 충청북도, 충청남도, 전라남도, 경상북도, 경상남도, 전북특별자치도

1. http://www.alimi.or.kr/dataview/a/selectAgriculturalMarketing.do
2. #areaId를 클릭 (시도 선택창)
3. 차례대로 #areaId > option:nth-child(2)부터 #areaId > option:nth-child(19)까지 선택 (시도 선택) 이때 택스트를 추출하여 "시도"칼럼에 저장장
4. 그 다음에 #list > tbody > tr:nth-child(1) > td > table > tbody > tr:nth-child(2) > td > table > tbody > tr > td > a > table > tbody > tr > td:nth-child(2) 클릭 (검색버튼)
5. 그러면 그에 맞는 결과 출력. 이때  #list > tbody > tr:nth-child(3) > td > table > tbody > tr:nth-child(2) 의 <td colspan="8">이면 데이터가 없다는 뜻이므로 다음 "시도"를 선택한다 (3번으로)
6. #list > tbody > tr:nth-child(3) > td > table > tbody > tr:nth-child(2) > td:nth-child(2) 는 "구분" 칼럼 (텍스트 추출)
7. #list > tbody > tr:nth-child(3) > td > table > tbody > tr:nth-child(2) > td.pl15는 "주소" 칼럼 (텍스트 추출)
8. #list > tbody > tr:nth-child(3) > td > table > tbody > tr:nth-child(2) > td:nth-child(4)는 "연간_임대료(원)" 칼럼 (텍스트 추출)
9. #list > tbody > tr:nth-child(3) > td > table > tbody > tr:nth-child(2) > td:nth-child(5)는 "면적(㎡)"칼럼 (텍스트 추출)
10. #list > tbody > tr:nth-child(3) > td > table > tbody > tr:nth-child(11) 보통 이런식으로 한 페이지에 데이터가 10개가 있지만, 10개가 아닐때도 있다.
11. 한 페이지에서 데이터를 다 받았으면 다음 페이지로 넘어간다. 처음 페이지는 "1페이지"이고 #list > tbody > tr:nth-child(4) > td > ul > a:nth-child(4) 이 버튼을 누르면 "2페이지"로 넘어간다. #list > tbody > tr:nth-child(4) > td > ul > a:nth-child(5)을 누르면 "3페이지"로 넘어간다. 
12. 페이지 양이 많으면 "10페이지"에서 #list > tbody > tr:nth-child(4) > td > ul > a:nth-child(13) 이 버튼을 넘으면 11~20페이지로 넘어간다. 만약 다음 11페이지부터 없거나, 이 페이지가 총 4페이지인 경우 #list > tbody > tr:nth-child(4) > td > ul > a:nth-child(11)을 눌러도 아무런 일이 일어나지 않는다. 이때 버튼의 요소는 <a href="#">이다. (위의 예시에서는 a:nth-child(13)인데 아래의 예시에서는 a:nth-child(11)이다. 그 이유는 총 현재 화면에서의 총 페이지의 숫자가 다르기 때문이다. 이 버튼은 #list > tbody > tr:nth-child(4) > td > ul > a의 버튼중 뒤에서 두번째에 있기 때문에 이걸로 구분 가능하다.) (예 : #list > tbody > tr:nth-child(4) > td > ul > a:nth-child(12)는 가장 마지막으로 가는 버튼이므로 그 앞의 #list > tbody > tr:nth-child(4) > td > ul > a:nth-child(11)버튼을 눌러야 한다.)
13. 결과 데이터는 데이터프레임으로 저장하여야 한다. 
14. 또한 오류로 인한 데이터 유실을 막기 위해 하나의 "시도"를 끝내고 다음 "시도"로 넘어가기 전에 encoding="cp949"로 csv파일을 저장한다.
15. WebDriverWait함수를 사용한다.

### 약 7분~8분 소요됨

gpt를 많이 활용하자

In [ ]:
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36'
options = webdriver.ChromeOptions()
options.add_argument(f"user-agent={user_agent}")

driver = webdriver.Chrome(options=options)
driver.get("http://www.alimi.or.kr/dataview/a/selectAgriculturalMarketing.do")

wait = WebDriverWait(driver, 10)

# 처음에 시도 옵션(텍스트와 value)을 미리 저장 (option:nth-child(2)부터 option:nth-child(19))
select_area_elem = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#areaId")))
select_area = Select(select_area_elem)
sido_options = []
for option in select_area.options[1:min(19, len(select_area.options))]:
    sido_text = option.text.strip()
    sido_value = option.get_attribute("value")
    sido_options.append((sido_text, sido_value))

results = []

# 저장된 sido_options를 순회 (sido에 "시도"명이 저장되어있다)
for sido, value in sido_options:
    print(f"현재 시도: {sido}")
    # DOM 갱신에 대비하여 시도 선택창을 매번 새로 찾음 (페이지가 바뀌면 HTML이 바뀔 수 있다)
    select_area_elem = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#areaId")))
    select_area = Select(select_area_elem)
    select_area.select_by_value(value)
    time.sleep(1)  # 화면 갱신 대기

    # 검색 버튼 클릭
    search_btn = wait.until(EC.element_to_be_clickable(
        (By.CSS_SELECTOR, "#list > tbody > tr:nth-child(1) > td > table > tbody > tr:nth-child(2) > td > table > tbody > tr > td > a > table > tbody > tr > td:nth-child(2)")
    ))
    search_btn.click()
    time.sleep(1)  # 검색 결과 로드 대기

    # 결과 테이블 tr:nth-child(2)의 td에 colspan="8"이면 데이터 없음 -> 다음 "시도"로 넘어감
    try:
        table_body = wait.until(EC.presence_of_element_located(
            (By.CSS_SELECTOR, "#list > tbody > tr:nth-child(3) > td > table > tbody")
        ))
        rows = table_body.find_elements(By.TAG_NAME, "tr")
        if len(rows) >= 2:
            cell = rows[1].find_element(By.TAG_NAME, "td")
            if cell.get_attribute("colspan") == "8":
                print(f"[{sido}] 검색결과 없음, 다음 시도로 이동")
                continue
        else:
            #데이터가 없을 경우우
            continue
    except Exception as e:
        #오류가 발생했을 때
        continue

    current_page = 1 #항상 첫번째 페이지부터 시작
    while True:
        print(f"[{sido}] {current_page}페이지") #현제 몇페이지인지 출력
        # 결과 테이블
        table_body = wait.until(EC.presence_of_element_located(
            (By.CSS_SELECTOR, "#list > tbody > tr:nth-child(3) > td > table > tbody")
        ))
        rows = table_body.find_elements(By.CSS_SELECTOR, "tr")

        for row in rows[1:]:
            cells = row.find_elements(By.TAG_NAME, "td")
            if len(cells) < 5 or (cells[0].get_attribute("colspan") and cells[0].get_attribute("colspan") == "8"):
                continue
            classification = cells[1].text.strip()  # "구분"
            try:
                address = row.find_element(By.CSS_SELECTOR, "td.pl15").text.strip()  # "주소"
            except Exception:
                address = ""
            rent = cells[3].text.strip()         # "연간_임대료(원)"
            area_val = cells[4].text.strip()       # "면적(㎡)"
            jimok = cells[5].text.strip()       # "지목"
            results.append({
                "시도": sido,
                "구분": classification,
                "주소": address,
                "연간_임대료(원)": rent,
                "면적(㎡)": area_val,
                "지목": jimok
            })

        # 페이지 네비게이션 처리 (다음 페이지로 넘어가는 버튼튼)
        pagination_buttons = driver.find_elements(By.CSS_SELECTOR, "#list > tbody > tr:nth-child(4) > td > ul > a")
        if len(pagination_buttons) < 2:
            print(f"[{sido}] 페이지 네비게이션 버튼이 부족하여 종료합니다.")
            break

        next_page_btn = None
        # 현재 페이지 번호+1에 해당하는 숫자 버튼 탐색
        for btn in pagination_buttons:
            if btn.text.strip() == str(current_page + 1):
                next_page_btn = btn
                break
        # 만약 숫자 버튼이 없으면 "다음" 혹은 ">" 버튼을 탐색
        if next_page_btn is None:
            for btn in pagination_buttons:
                if btn.text.strip() in ["다음", ">"]:
                    next_page_btn = btn
                    break

        if next_page_btn is None:
            #다음 페이지 버튼이 없을 때
            break

        # href가 "#"이거나 텍스트가 비어있으면 더 이상 이동할 수 없음
        if next_page_btn.get_attribute("href") == "#" or next_page_btn.text.strip() == "":
            print(f"[{sido}] 다음 페이지 버튼이 비활성화되어 현재 페이지 {current_page}에서 종료합니다.")
            break

        # 현재 테이블 내용 저장 (페이지 변화 확인용)
        prev_table_html = table_body.get_attribute("innerHTML")
        
        try:
            next_page_btn.click()
        except Exception as e:
            #오류가 발생했을 때 
            break

        time.sleep(1)
        # 새 페이지의 테이블 로딩 후 내용 비교
        try:
            new_table_body = wait.until(EC.presence_of_element_located(
                (By.CSS_SELECTOR, "#list > tbody > tr:nth-child(3) > td > table > tbody")
            ))
        except Exception as e:
            #오류가 발생했을 때
            break
        new_table_html = new_table_body.get_attribute("innerHTML")
        if prev_table_html == new_table_html:
            print(f"[{sido}] 페이지 내용이 동일하여 다음 페이지가 없는 것으로 판단.") 
            #다음 페이지 버튼을 눌렀을 경우, 마지막 페이지의 내용이 유지된다
            break

        current_page += 1 #다음 페이지로

    # csv파일 저장
    df = pd.DataFrame(results)
    df.to_csv("결과데이터/agricultural_marketing_data.csv", index=False, encoding="cp949")
    print(f"[{sido}] 데이터 저장 완료 (agricultural_marketing_data.csv).")

driver.quit()


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=134.0.6998.179)
Stacktrace:
	GetHandleVerifier [0x00007FF6B7EE4C25+3179557]
	(No symbol) [0x00007FF6B7B488A0]
	(No symbol) [0x00007FF6B79D91CA]
	(No symbol) [0x00007FF6B79B1753]
	(No symbol) [0x00007FF6B7A5F83E]
	(No symbol) [0x00007FF6B7A7FBE2]
	(No symbol) [0x00007FF6B7A57A03]
	(No symbol) [0x00007FF6B7A206D0]
	(No symbol) [0x00007FF6B7A21983]
	GetHandleVerifier [0x00007FF6B7F467CD+3579853]
	GetHandleVerifier [0x00007FF6B7F5D1D2+3672530]
	GetHandleVerifier [0x00007FF6B7F52153+3627347]
	GetHandleVerifier [0x00007FF6B7CB092A+868650]
	(No symbol) [0x00007FF6B7B52FFF]
	(No symbol) [0x00007FF6B7B4F4A4]
	(No symbol) [0x00007FF6B7B4F646]
	(No symbol) [0x00007FF6B7B3EAA9]
	BaseThreadInitThunk [0x00007FF94B24E8D7+23]
	RtlUserThreadStart [0x00007FF94C0BBF6C+44]


In [305]:
df

,시도,구분,주소,연간_임대료(원),면적(㎡),지목
0,부산광역시,임대-수탁,부산광역시 기장군 기장읍 만화리 352-2,"200,000","1,755",답
1,부산광역시,임대-수탁,부산광역시 기장군 정관읍 예림리 2,"2,000,000","3,478",답
2,부산광역시,임대-수탁,부산광역시 기장군 정관읍 월평리 108-26,"50,000",471,답
3,부산광역시,임대-수탁,부산광역시 기장군 정관읍 월평리 108-25,"250,000",946,답
4,대구광역시,임대-수탁,대구광역시 군위군 군위읍 하곡리 40,"300,000","1,133",답
...,...,...,...,...,...,...
2345,전북특별자치도,매도-맞춤형,전북특별자치도 정읍시 신태인읍 양괴리 1515,"120,540,000","3,985",답
2346,전북특별자치도,매도-맞춤형,전북특별자치도 정읍시 영원면 앵성리 1105,"150,000,000","3,942",답
2347,전북특별자치도,매도-경영회생,전북특별자치도 진안군 주천면 대불리 801-1,"40,414,000","3,749",전
2348,전북특별자치도,임대-수탁,전북특별자치도 진안군 주천면 운봉리 197,"320,000","2,519",답


잘 크롤링되었다.

## 전처리

In [315]:
df.isna().sum()

시도           0
구분           0
주소           0
연간_임대료(원)    0
면적(㎡)        0
지목           0
dtype: int64

In [316]:
df.dtypes

시도           object
구분           object
주소           object
연간_임대료(원)    object
면적(㎡)        object
지목           object
dtype: object

연간_임대료(원), 면적(㎡)에 쉼표를 제거하고 숫자형식으로 바꾸자

In [321]:
df["연간_임대료(원)"] = df["연간_임대료(원)"].astype(str).str.replace(",", "").astype(float)
df["면적(㎡)"] = df["면적(㎡)"].astype(str).str.replace(",", "").astype(float)

In [322]:
df.dtypes

시도            object
구분            object
주소            object
연간_임대료(원)    float64
면적(㎡)        float64
지목            object
dtype: object

## 주소 분할

In [22]:
df[["시도", "시군구", "읍면동", "상세주소"]] = df["주소"].apply(split_full_address)

In [23]:
df.시도.unique()

array(['부산광역시', '대구광역시', '인천광역시', '광주광역시', '울산광역시', '세종특별자치시', '경기도',
       '충청북도', '충청남도', '전라남도', '경상북도', '경상남도', '제주특별자치도', '전북특별자치도'],
      dtype=object)

In [30]:
df.isna().sum()

County            0
City              0
Town              0
Address           0
Annual_Rent(₩)    0
Area(㎡)           0
Land_Type         0
dtype: int64

In [26]:
df=df[["시도","시군구","읍면동","상세주소","연간_임대료(원)", "면적(㎡)", "지목"]]
df.head(1)

,시도,시군구,읍면동,상세주소,연간_임대료(원),면적(㎡),지목
0,부산광역시,기장군,기장읍,만화리 352-2,200000.0,1755.0,답


In [ ]:
df.columns=["County","City","Town","Address","Annual_Rent","Area","Land_Type"]
df.head(1)

,County,City,Town,Address,Annual_Rent(₩),Area(㎡),Land_Type
0,부산광역시,기장군,기장읍,만화리 352-2,200000.0,1755.0,답


## 저장

In [ ]:
df.to_csv("결과데이터/전국농지매물정보.csv", index=False, encoding="utf-8-sig")